<a href="https://colab.research.google.com/github/ARoDias/DataMiningProject_CTCT_2024Analysis/blob/main/Feature_selection_and_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature_selection and Clustering notebook

This notebook answers the following research question:
* **RQ4:** What distinct clusters are evident in student performances based on different types of assessment points?

And selects features for the following research question:
* **RQ5:** Can student performance in one theme of the CTCT course predict outcomes in subsequent themes?

## Important:
Create a folder called **inputs** and store files there to load.


# 1. Importing Libraries

In [ ]:
# Data Manipulation Libraries
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical operations on large multi-dimensional arrays and matrices

# Visualization Libraries
import matplotlib.pyplot as plt  # Plotting library for creating static, animated, and interactive visualizations

# Machine Learning - Preprocessing and Model Evaluation
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif  # Feature selection
from sklearn.ensemble import RandomForestClassifier  # Classification using Random Forests
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, KBinsDiscretizer, StandardScaler  # Preprocessing tools
from sklearn.impute import SimpleImputer  # Missing value imputation
from sklearn.tree import DecisionTreeClassifier  # Classification using Decision Trees
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score  # Clustering evaluation metrics
from sklearn.cluster import KMeans, AgglomerativeClustering  # Clustering algorithms
from sklearn.decomposition import PCA  # Principal Component Analysis for dimensionality reduction

# Additional Imports for Regression Feature Selection
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_classif, mutual_info_regression  # Feature selection for regression using F-test
from sklearn.ensemble import RandomForestRegressor  # Regression using Random Forests
from sklearn.linear_model import Lasso  # Lasso regression for feature selection
from sklearn.tree import DecisionTreeRegressor  # Regression using Decision Trees

# Hierarchical Clustering Performance Enhancement
import scipy.cluster.hierarchy as sch  # Additional hierarchical clustering methods

# Integration with Google Colaboratory
from google.colab import files  # File upload and download functionality for Google Colab
from pathlib import Path  # Object-oriented filesystem paths

# Document Generation
!pip install python-docx  # Install python-docx for creating and updating Microsoft Word (.docx) files
from docx import Document  # Library for creating and updating Microsoft Word (.docx) files

# Date and Time Utilities
from datetime import datetime  # Utilities for date and time manipulation

# Data Serialization
import pickle  # Serialization and deserialization of Python objects
import os  # Operating system dependent functionality
from io import StringIO  # In-memory file-like object for text I/O
import shutil  # High-level file operations like copying and removal

# Display Utilities for Jupyter Notebooks
from IPython.display import display as ipy_display  # Display utilities for Jupyter notebooks

print("Libraries imported successfully.")

if not os.path.exists('inputs'):
    os.makedirs('inputs')

Libraries imported successfully.


# 2. Auxiliary Methods

## 2.1 For loading files, data verification and preparation:

###### load_compose_feather_files

In [ ]:
# Function to load Feather files with only numbers in the filename from a specified directory, excluding keys from datasets
def load_compose_feather_files(datasets, path='inputs'):
    # Get a list of Feather files in the directory with only numbers in the filename
    files = [f for f in os.listdir(path) if f.endswith('.feather') and f[:-8].isdigit() and not f.startswith('dataset_')]
    compose_datasets = {}

    # Load the compose datasets
    for file in files:
        # Extract the index from the file name
        index = int(file.replace('.feather', ''))
        # Check if the index is already in datasets to avoid duplication
        if index not in datasets:
            # Read the Feather file into a DataFrame
            compose_datasets[index] = pd.read_feather(os.path.join(path, file))

    return compose_datasets

######  ensure_unique_columns

In [ ]:
def ensure_unique_columns(df):
    # Append the column name with an increasing count if duplicate column names are found
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df


######  combine_datasets

In [ ]:
def combine_datasets(datasets):
    combined_df = pd.DataFrame()

    for year, df in datasets.items():
        # Ensure the DataFrame index is unique
        df = df.reset_index(drop=True)
        # Ensure the DataFrame columns are unique
        df = ensure_unique_columns(df)
        # Add a 'Year' column to identify the year
        df['Year'] = year
        # Concatenate the DataFrame to the combined DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    return combined_df

###### Update Dataset and Dictionary of Lists with IDs of activities
update_column_names(dataset, activity_dict, sync_columns) & update_all_lists(all_lists, year, column_mapping)

In [ ]:
# Function to update column names based on a mapping
def update_column_names(dataset, activity_dict, sync_columns):
    """
    Updates the column names of the dataset based on a mapping from the activity dictionary.

    Args:
    - dataset (pd.DataFrame): The dataset whose columns need to be updated.
    - activity_dict (dict): Dictionary containing mappings of old column names to new names and additional info.
    - sync_columns (list): List of columns that need to be checked and potentially updated.

    Returns:
    - dataset (pd.DataFrame): The dataset with updated column names.
    - column_mapping (dict): A dictionary mapping original column names to their updated names.

    Functionality:
    - For each column in `sync_columns`, it checks if there is a corresponding entry in `activity_dict`.
    - Depending on the type of activity, the column name is updated to its mapped value or retained as is.
    - The updated column names are applied to the dataset.
    """
    column_mapping = {}
    for col in sync_columns:
        if col in activity_dict:
            activity_info = activity_dict[col]
            activity_type = activity_info[2]
            if activity_type in ['A', 'B', 'C', 'SPA']:
                column_mapping[col] = str(activity_info[0])
            else:
                column_mapping[col] = col  # Keep the original column if not evaluated
        else:
            column_mapping[col] = col  # Keep the original column if not in the dictionary

    # Apply the mapping to the columns
    new_columns = [column_mapping.get(col, col) for col in dataset.columns]
    dataset.columns = new_columns
    return dataset, column_mapping

# Function to update all relevant lists with the new column names
def update_all_lists(all_lists, year, column_mapping):
    """
    Updates all relevant lists in the `all_lists` dictionary to reflect the updated column names.

    Args:
    - all_lists (dict): The dictionary containing various lists of columns for each dataset year.
    - year (int or str): The year corresponding to the dataset being updated.
    - column_mapping (dict): A dictionary mapping original column names to their updated names.

    Returns:
    - updated_lists (dict): The updated dictionary with all lists reflecting the new column names.

    Functionality:
    - The function iterates through various lists in `all_lists` and updates each list's column names based on `column_mapping`.
    - It handles different categories of lists, including basic lists, weekly and participation lists, activity type lists, and created columns.
    """
    updated_lists = all_lists[str(year)].copy()

    # Update basic lists
    for list_name in updated_lists['basic_lists']:
        updated_lists['basic_lists'][list_name] = [column_mapping.get(col, col) for col in updated_lists['basic_lists'][list_name]]

    # Update weekly and participation lists
    for list_name in updated_lists['weekly_and_participation_lists']:
        if isinstance(updated_lists['weekly_and_participation_lists'][list_name], dict):
            updated_lists['weekly_and_participation_lists'][list_name] = {
                k: [column_mapping.get(col, col) for col in v]
                for k, v in updated_lists['weekly_and_participation_lists'][list_name].items()
            }
        else:
            updated_lists['weekly_and_participation_lists'][list_name] = [
                column_mapping.get(col, col) for col in updated_lists['weekly_and_participation_lists'][list_name]
            ]

    # Update activity type lists
    for list_name in updated_lists['activity_type_lists']:
        updated_lists['activity_type_lists'][list_name] = [
            column_mapping.get(col, col) for col in updated_lists['activity_type_lists'][list_name]
        ]

    # Update created columns lists
    for list_name in updated_lists['created_columns']:
        updated_lists['created_columns'][list_name] = [
            column_mapping.get(col, col) for col in updated_lists['created_columns'][list_name]
        ]

    return updated_lists

## 2.2 For notebook output manipulation:

###### print_to_doc

In [ ]:
# Custom print function to capture output to a Word document
def print_to_doc(*args, **kwargs):
    """
    This function captures the output and appends it to a Word document.

    Args:
    - *args: Variable length argument list. The arguments will be concatenated into a single string.
    - **kwargs: Arbitrary keyword arguments. This function does not use kwargs directly, but allows them for flexibility.

    Functionality:
    - The function joins all positional arguments into a single string separated by spaces.
    - The resulting text is added as a new paragraph to the global Word document object `doc`.
    """
    text = ' '.join(map(str, args))
    doc.add_paragraph(text)

###### custom_display

In [ ]:
# Custom display function to capture output to a Word document
def custom_display(*args, **kwargs):
    """
    This function captures display output and either saves it to a Word document or displays it using IPython's display function.

    Args:
    - *args: Variable length argument list. These are the objects to be displayed or saved.
    - **kwargs: Arbitrary keyword arguments. These are passed through to the display function if needed.

    Functionality:
    - If `output_to_docx` is set to 1, the function saves the display output to the Word document:
        - If the argument is a DataFrame, it converts it to a string and captures the output.
        - Otherwise, it converts the argument to a string directly.
        - The resulting text is added as a new paragraph to the global Word document object `doc`.
    - If `output_to_docx` is not set to 1, it uses the IPython display function (`ipy_display`) to show the output as usual.
    """
    if output_to_docx == 1:
        for arg in args:
            if isinstance(arg, pd.DataFrame):
                # Convert DataFrame to a string representation for saving
                buf = StringIO()
                arg.to_string(buf)
                text = buf.getvalue()
            else:
                # Convert other types of arguments to string
                text = str(arg)
            doc.add_paragraph(text)
    else:
        # Display using IPython's display function
        ipy_display(*args, **kwargs)

###### save_document

In [ ]:
# Function to save the document with the current date
def save_document(filename='feature_selection.docx'):
    """
    Saves the current Word document with a specified filename and appends the current date and time.

    Args:
    - filename (str): The name of the file to save the document as. Defaults to 'feature_selection.docx'.

    Functionality:
    - Appends a paragraph with the current date and time to the document.
    - Saves the document to the specified filename.
    - Prints a confirmation message to the console with the filename.
    """
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    doc.add_paragraph(f"Document saved on {current_date}")
    doc.save(filename)
    print(f"Document saved as {filename}")

###### combine_selected_features

In [ ]:
def combine_selected_features(selected_features):
    """
    Combine selected features across all years into a single set for each method.

    Args:
    - selected_features (dict): Dictionary of selected features for each year and method.

    Returns:
    - combined_features (dict): Combined features for each method across all years.
    """
    combined_features = {}
    for year_features in selected_features.values():
        for method, features in year_features.items():
            if method not in combined_features:
                combined_features[method] = set(features)
            else:
                combined_features[method].update(features)

    # Convert sets to lists
    combined_features = {method: list(features) for method, features in combined_features.items()}
    return combined_features

###### save_combined_features_to_feather

In [ ]:
def save_combined_features_to_feather(selected_features, combined_df, target_column, output_dir, filename_suffix, exclusion_type):
    """
    Save the combined selected features to a Feather file with the target column as the last column,
    and print information about the DataFrame being saved.

    Args:
    - selected_features (dict): Dictionary of combined selected features for each method.
    - combined_df (pd.DataFrame): The combined dataset containing all features.
    - target_column (str): The target column to be included as the last column.
    - output_dir (str or Path): Directory where the Feather files should be saved.
    - filename_suffix (str): Suffix to append to the filenames.
    - exclusion_type (str): The type of exclusion used for this selection.

    Returns:
    - None
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    for method, features in selected_features.items():
        if features:
            # Ensure the target column is not in the features list
            features = [feature for feature in features if feature != target_column]

            # Append the target column at the end
            features_with_target = features + [target_column]

            # Check if all selected features are in the combined_df
            missing_features = [f for f in features_with_target if f not in combined_df.columns]
            if missing_features:
                print(f"Warning: The following features are missing in combined_df for method {method}: {missing_features}")
                features_with_target = [f for f in features_with_target if f in combined_df.columns]

            # Subset the DataFrame with the selected features
            df_selected = combined_df[features_with_target]

            # Ensure the target column is the last column
            if df_selected.columns[-1] != target_column:
                print(f"Adjusting column order to ensure {target_column} is the last column.")
                df_selected = df_selected[features + [target_column]]

            # Print information about the DataFrame being saved
            print(f"Saving DataFrame for method '{method}' with exclusion type '{exclusion_type}':")
            print(f"- Number of features: {len(features)}")
            print(f"- Shape of DataFrame: {df_selected.shape}")
            print(f"- First few columns: {df_selected.columns[:5].tolist()}...")

            # Construct the output path using Path object
            output_path = output_dir / f"{filename_suffix}_{method}_{exclusion_type}.feather"

            # Save to Feather file
            df_selected.reset_index(drop=True).to_feather(output_path)

            # Verify that the file was saved correctly
            loaded_df = pd.read_feather(output_path)
            if loaded_df.columns[-1] == target_column:
                print(f"Success: Saved {output_path} with {target_column} as the last column.")
            else:
                print(f"Error: {target_column} is not the last column in {output_path}. Last column is {loaded_df.columns[-1]}")
        else:
            print(f"No features selected for method {method}, skipping Feather file creation.")


###### save_selected_features_to_feather

In [ ]:
def save_selected_features_to_feather(selected_features, combined_df, target_column, output_dir, filename_suffix, exclusion_type):
    """
    Save the selected features to Feather files with the target column as the last column.

    Args:
    - selected_features (dict): Dictionary of selected features for each method.
    - combined_df (pd.DataFrame): The combined dataset containing all features.
    - target_column (str): The target column to be included as the last column.
    - output_dir (str or Path): Directory where the Feather files should be saved.
    - filename_suffix (str): Suffix to append to the filenames.
    - exclusion_type (str): The type of exclusion used for this selection.

    Returns:
    - None
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    for method, features in selected_features.items():
        if features:
            # Ensure the target column is not in the features list
            features = [feature for feature in features if feature != target_column]

            # Append the target column at the end
            features_with_target = features + [target_column]

            # Check if all selected features are in the combined_df
            missing_features = [f for f in features_with_target if f not in combined_df.columns]
            if missing_features:
                print(f"Warning: The following features are missing in combined_df for method {method}: {missing_features}")
                features_with_target = [f for f in features_with_target if f in combined_df.columns]

            # Subset the DataFrame with the selected features
            df_selected = combined_df[features_with_target]

            # Ensure the target column is the last column
            if df_selected.columns[-1] != target_column:
                print(f"Adjusting column order to ensure {target_column} is the last column.")
                df_selected = df_selected[features + [target_column]]

            # Construct the output path using Path object
            output_path = output_dir / f"{filename_suffix}_{method}_{exclusion_type}.feather"

            # Save to Feather file
            df_selected.reset_index(drop=True).to_feather(output_path)

            # Verify that the file was saved correctly
            loaded_df = pd.read_feather(output_path)
            if loaded_df.columns[-1] == target_column:
                print(f"Success: Saved {output_path} with {target_column} as the last column.")
            else:
                print(f"Error: {target_column} is not the last column in {output_path}. Last column is {loaded_df.columns[-1]}")
        else:
            print(f"No features selected for method {method}, skipping Feather file creation.")

###### create_and_save_datasets

In [ ]:
def create_and_save_datasets(combined_df, exclude_columns, target_column, output_dir, filename_suffix="combined_df"):
    """
    Create a dataset with selected features (excluding specified columns) and save it to a Feather file.

    Args:
    - combined_df (pd.DataFrame): The combined dataset containing all features.
    - exclude_columns (dict): Dictionary where keys are years and values are sets of columns to exclude.
    - target_column (str): The target column to be included as the last column.
    - output_dir (str or Path): Directory where the Feather files should be saved.
    - filename_suffix (str): Suffix to append to the filenames.

    Returns:
    - None
    """
    # Ensure output_dir is a Path object
    output_dir = Path(output_dir)

    # Ensure the output directory exists
    output_dir.mkdir(parents=True, exist_ok=True)

    # Flatten all columns to exclude across all years
    all_columns_to_exclude = set()
    for columns in exclude_columns.values():
        all_columns_to_exclude.update(columns)

    # Select all columns except those to be excluded
    selected_features = [col for col in combined_df.columns if col not in all_columns_to_exclude]

    # Ensure target column is the last column
    if target_column in selected_features:
        selected_features.remove(target_column)
    selected_features.append(target_column)

    # Subset the DataFrame with the selected features
    df_selected = combined_df[selected_features]

    # Construct the output path using Path object and include target_column in the filename
    output_path = output_dir / f"{filename_suffix}_{target_column}.feather"

    # Save to Feather file
    df_selected.reset_index(drop=True).to_feather(output_path)
    print(f"Saved selected features to {output_path}.")



###### print_activity_details_for_year

In [ ]:
def print_activity_details_for_year(dataset_params, year, activity_keys):
    """
    Print details of specified activities for a given year.

    Args:
    - dataset_params (dict): A dictionary containing parameters for datasets, where each key represents a year.
    - year (int or str): The year for which activity details are to be printed.
    - activity_keys (list): A list of keys identifying specific activities within the activity dictionary.

    Functionality:
    - The function checks if the specified year exists in the dataset parameters.
    - If the year exists, it retrieves the activity dictionary for that year and iterates over the provided activity keys.
    - For each key, it checks if the key is present in the activity dictionary:
        - If found, it prints the activity details associated with the key.
        - If not found, it prints a message indicating that the activity was not found.
    - If the year is not found in `dataset_params`, it prints a message stating that no data is available for that year.
    """
    if year in dataset_params:
        activity_dict = dataset_params[year].get('activity_dictionary', {})
        print(f"Activities for {year}:")
        for key in activity_keys:
            if key in activity_dict:
                activity_data = activity_dict[key]
                print(f"  Activity {key}: {activity_data}")
            else:
                print(f"  Activity {key}: Not found")
    else:
        print(f"No data available for the year {year}")

######  get_activity_index

In [ ]:
def get_activity_index(dataset_params, year, activity_key):
    """
    Retrieve the index of a specific activity for a given year.

    Args:
    - dataset_params (dict): A dictionary containing parameters for datasets, where each key represents a year.
    - year (int or str): The year for which the activity index is to be retrieved.
    - activity_key (str): The key identifying the specific activity within the activity dictionary.

    Returns:
    - The index of the activity if found, or None if the activity or year is not present in the dataset.

    Functionality:
    - The function attempts to retrieve the activity index from the activity dictionary for the specified year.
    - It uses a try-except block to catch any `KeyError` that occurs if the year or activity key is not found.
    - If the key exists, it returns the index (typically the first element of the activity data).
    - If not found, it returns None.
    """
    try:
        return dataset_params[year]['activity_dictionary'][activity_key][0]
    except KeyError:
        return None

###### get_activity_name

In [ ]:
def get_activity_name(dataset_params, year, activity_key):
    """
    Retrieve the name of a specific activity for a given year.

    Args:
    - dataset_params (dict): A dictionary containing parameters for datasets, where each key represents a year.
    - year (int or str): The year for which the activity name is to be retrieved.
    - activity_key (str): The key identifying the specific activity within the activity dictionary.

    Returns:
    - The name of the activity if found, or None if the activity or year is not present in the dataset.

    Functionality:
    - The function attempts to retrieve the activity name from the activity dictionary for the specified year.
    - It uses a try-except block to catch any `KeyError` that occurs if the year or activity key is not found.
    - If the key exists, it returns the name (typically the second element of the activity data).
    - If not found, it returns None.
    """
    try:
        return dataset_params[year]['activity_dictionary'][activity_key][1]
    except KeyError:
        return None

###### get_activity_name_by_id

In [ ]:
def get_activity_name_by_id(activity_id, dataset_params, year):
    """
    Retrieve the name of an activity by its ID for a given year.

    Args:
    - activity_id (int or str): The ID of the activity to find.
    - dataset_params (dict): A dictionary containing parameters for datasets, where each key represents a year.
    - year (int or str): The year for which the activity name is to be retrieved.

    Returns:
    - The name of the activity corresponding to the given ID, or None if not found.

    Functionality:
    - The function iterates over the items in the activity dictionary for the specified year.
    - It checks if the ID of each activity matches the provided `activity_id`.
    - If a match is found, it returns the name of the activity (the second element in the dictionary entry).
    - If no match is found, it returns None.
    """
    for value in dataset_params[str(year)]['activity_dictionary'].values():
        if value[0] == activity_id:
            return value[1]
    return None

###### print_activity_details_for_year_by_id

In [ ]:
def print_activity_details_for_year_by_id(dataset_params, year, activity_ids):
    """
    Print details of activities for a given year based on their IDs.

    Args:
    - dataset_params (dict): A dictionary containing parameters for datasets, where each key represents a year.
    - year (int or str): The year for which activity details are to be printed.
    - activity_ids (list): A list of IDs identifying specific activities within the activity dictionary.

    Functionality:
    - The function checks if the specified year exists in the dataset parameters.
    - If the year exists, it retrieves the activity dictionary for that year and iterates over the provided activity IDs.
    - For each ID, it uses `get_activity_name_by_id` to find the corresponding activity name:
        - If found, it prints the activity ID and name.
        - If not found, it prints a message indicating that the activity ID was not found.
    - If the year is not found in `dataset_params`, it prints a message stating that no data is available for that year.
    """
    if str(year) in dataset_params:
        activity_dict = dataset_params[str(year)].get('activity_dictionary', {})
        print(f"Activities for {year}:")
        for activity_id in activity_ids:
            activity_name = get_activity_name_by_id(int(activity_id), activity_dict, int(year))
            if activity_name:
                print(f"  Activity ID {activity_id}: {activity_name}")
            else:
                print(f"  Activity ID {activity_id}: Not found")
    else:
        print(f"No data available for the year {year}")

###### check_missing_values

In [ ]:
def check_missing_values(datasets):
    """
    Check and print the missing values for each dataset in a collection of datasets.

    Args:
    - datasets (dict): A dictionary where keys are years (or other identifiers) and values are DataFrames representing datasets.

    Functionality:
    - The function iterates over each dataset in the `datasets` dictionary.
    - For each dataset, it calculates the sum of missing values for each column.
    - If there are columns with missing values, it prints the count of missing values per column.
    """
    for key, dataset in datasets.items():
        print(f"Missing values for {key} dataset:")
        missing_values = dataset.isnull().sum()
        # Filter and print only columns with missing values
        print(missing_values[missing_values > 0])

### Convert activity IDs to names (from lists)


###### convert_feature_ids_to_activity_names

In [ ]:
def convert_feature_ids_to_activity_names(selected_features, dataset_params):
    """
    Convert feature IDs to activity names using the provided dataset parameters.

    Args:
    - selected_features (dict or pd.Index): A dictionary of selected features for each year and method, or an Index object.
    - dataset_params (dict): A dictionary containing dataset parameters, including activity dictionaries for each year.

    Returns:
    - converted_features (dict or list): A dictionary with feature IDs converted to activity names, or a list if input is an Index.

    Functionality:
    - The function checks if `selected_features` is a dictionary or an Index.
    - If it's a dictionary, it iterates over each year and method, converting feature IDs to activity names using the dataset parameters.
    - If it's an Index, it assumes the selected features are for a single year and converts them similarly.
    """
    def get_activity_name_by_id(activity_id, dataset_params, year):
        """
        Helper function to get the activity name given its ID, year, and dataset parameters.
        """
        for value in dataset_params[str(year)]['activity_dictionary'].values():
            if str(value[0]) == str(activity_id):
                return value[1]
        return None

    if isinstance(selected_features, dict):
        converted_features = {}
        for year, methods in selected_features.items():
            if str(year) not in dataset_params:
                print(f"Year {year} not found in dataset_params.")
                continue

            converted_features[year] = {}

            for method, features in methods.items():
                converted_names = []
                for feature in features:
                    if feature.isdigit():
                        activity_name = get_activity_name_by_id(int(feature), dataset_params, int(year))
                        if activity_name:
                            converted_names.append(activity_name)
                        else:
                            converted_names.append(f"ID_{feature}_Not_Found")
                    else:
                        converted_names.append(feature)
                converted_features[year][method] = converted_names

        return converted_features

    elif isinstance(selected_features, pd.Index):
        converted_names = []
        year = list(dataset_params.keys())[0]  # Assumes first year if multiple years exist
        for feature in selected_features:
            if feature.isdigit():
                activity_name = get_activity_name_by_id(int(feature), dataset_params, int(year))
                if activity_name:
                    converted_names.append(activity_name)
                else:
                    converted_names.append(f"ID_{feature}_Not_Found")
            else:
                converted_names.append(feature)
        return converted_names

###### convert_feature_ids_to_activity_names_complete

In [ ]:
def convert_feature_ids_to_activity_names_complete(selected_features, dataset_params):
    """
    Convert feature IDs to activity names using the dataset parameters for multiple years and methods.

    Args:
    - selected_features (dict): A dictionary of selected features for each method.
    - dataset_params (dict): A dictionary containing parameters and activity dictionaries for each year.

    Returns:
    - converted_features (dict): A dictionary with converted feature names.

    Functionality:
    - The function extracts activity dictionaries for each year from `dataset_params`.
    - It iterates over each method and its features in `selected_features`.
    - For each feature, it searches through the activity dictionaries for a matching ID.
    - If found, it appends the activity name to `converted_features`. If not found, it appends 'Unknown'.
    """
    # Extract activity dictionaries from dataset_params
    activity_dictionaries = {
        str(year): params['activity_dictionary'] for year, params in dataset_params.items()
    }

    converted_features = {}

    # Iterate through each dataset (identified by key, such as year)
    for key, methods in selected_features.items():
        converted_features[key] = {}

        for method, features in methods.items():
            converted_names = []
            for feature in features:
                # Identify the year from the method string, e.g., 'RF_2022' -> '2022'
                year = method.split('_')[-1]
                activity_dict = activity_dictionaries.get(year, {})

                # Attempt to find the activity name by ID in the relevant activity dictionary
                activity_name = None
                for activity_info in activity_dict.values():
                    if str(activity_info[0]) == feature:
                        activity_name = activity_info[1]
                        break

                # Append the activity name or 'Unknown' if not found
                if activity_name:
                    converted_names.append(activity_name)
                else:
                    converted_names.append(f'Unknown (ID: {feature})')

            converted_features[key][method] = converted_names

    return converted_features

## 2.3 For filtering specific instances:

###### filter_missing_and_zero_final_grade

In [ ]:
def filter_missing_and_zero_final_grade(datasets):
    """
    Filter out rows with missing or zero values in the 'FinalGrade' column from each dataset.

    Args:
    - datasets (dict): A dictionary where keys are years (or other identifiers) and values are DataFrames representing datasets.

    Returns:
    - filtered_datasets (dict): A dictionary with the same structure as the input, but with datasets filtered.

    Functionality:
    - The function iterates over each dataset in `datasets`.
    - For each dataset, it drops rows where the 'FinalGrade' column is missing (NaN).
    - Then, it further filters out rows where 'FinalGrade' is zero.
    - It prints the number of rows before and after filtering, and the number of rows dropped.
    """
    filtered_datasets = {}
    for year, dataset in datasets.items():
        # Drop rows with NaN in 'FinalGrade' and where 'FinalGrade' is 0
        filtered_dataset = dataset.dropna(subset=['FinalGrade'])
        filtered_dataset = filtered_dataset[filtered_dataset['FinalGrade'] != 0]
        filtered_datasets[year] = filtered_dataset
        # Print the results of the filtering process
        print(f"After filtering, {year} dataset has {len(filtered_dataset)} rows. Before had {len(dataset)}. Dropped {len(dataset) - len(filtered_dataset)} instances/students.")
    return filtered_datasets

###### filter_dataset_by_activities

In [ ]:
def filter_dataset_by_activities(dataset, known_activities, target_column='FinalGradeInteger', additional_columns=None):
    """
    Filter the dataset to retain only specified target column, additional columns, and columns related to known activities.

    Args:
    - dataset (pd.DataFrame): The dataset to be filtered.
    - known_activities (list): List of tuples where each tuple contains an activity ID and its description.
    - target_column (str): The target column to retain in the dataset. Default is 'FinalGradeInteger'.
    - additional_columns (list): Additional columns to retain in the dataset. Default is None.

    Returns:
    - filtered_dataset (pd.DataFrame): The dataset filtered to include only the specified columns.

    Functionality:
    - The function extracts activity IDs from `known_activities`.
    - It constructs a list of columns to retain, including the target column, additional columns, and any columns in the dataset matching the activity IDs.
    - The dataset is then filtered to include only these columns and returned.
    """
    if additional_columns is None:
        additional_columns = []

    # Extract activity IDs from the known_activities list
    activity_ids = [activity_id for activity_id, _ in known_activities]

    # Create a list of columns to retain
    columns_to_retain = [target_column] + additional_columns + [col for col in dataset.columns if col in activity_ids]

    # Filter the dataset
    filtered_dataset = dataset[columns_to_retain]

    return filtered_dataset

###### filter_known_activities

In [ ]:
def filter_known_activities(activity_dict):
    """
    Filter out activities with 'Unknown' as their description from a given activity dictionary.

    Args:
    - activity_dict (dict): A dictionary where keys are method names (e.g., 'MI', 'RF', 'DT') and values are lists of tuples.
                            Each tuple contains an activity ID and its description.

    Returns:
    - known_activities (list): A list of tuples with activity IDs and descriptions, excluding those with 'Unknown'.

    Functionality:
    - The function iterates over each method and its activities in `activity_dict`.
    - It appends activities that do not have 'Unknown' as their description to the `known_activities` list.
    - The result is a list of activities that have valid descriptions.
    """
    known_activities = []
    for activities in activity_dict.values():
        for activity_id, activity_desc in activities:
            if activity_desc != 'Unknown':
                known_activities.append((activity_id, activity_desc))
    return known_activities

# 3. Load Files

## 3.1. Change Printing Mode

In [ ]:
# Change Printing Mode
output_to_docx = 0 # Control output (0 for notebook, 1 for docx)

In [ ]:
# Load Files

# Initialize a Word document to capture output
doc = Document()

if output_to_docx == 1:
    print = print_to_doc
    display = custom_display

# Ensure Charts directory exists
if not os.path.exists('Charts'):
    os.makedirs('Charts')

output_dir = 'output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
os.makedirs('output/feather_files', exist_ok=True)

# Load datasets and other files
dataset_years = [2016, 2017, 2018, 2019, 2022, 2023, 2024]
datasets = {}

for year in dataset_years:
    file_path = f'inputs/dataset_{year}.feather'
    datasets[year] = pd.read_feather(file_path)

# Load the additional compose datasets
compose_datasets = load_compose_feather_files(datasets)

activities = pd.read_feather('inputs/activities.feather')
activities2 = pd.read_feather('inputs/activities2.feather')
summary = pd.read_feather('inputs/summary.feather')

# Load additional lists and parameters
with open('inputs/all_lists.pkl', 'rb') as file:
    all_lists = pickle.load(file)
with open('inputs/all_lists_id.pkl', 'rb') as file:
    all_lists_id = pickle.load(file)
with open('inputs/dataset_params.pkl', 'rb') as file:
    dataset_params = pickle.load(file)

file_path = f'inputs/dataset_no_missing.feather'
dataset_complete = pd.read_feather(file_path)

print("Datasets and other files loaded successfully.")

Datasets and other files loaded successfully.


In [ ]:
doc.add_heading('Information about the loaded datasets', 1)

In [ ]:
doc.add_heading('dataset_complete', 2)
print("dataset_complete = the loaded combined dataset with all years removing columns with with all missing values atored in inputs/dataset_no_missing.feather file resulting from data_preparation")
dataset_complete.info()

dataset_complete = the loaded combined dataset with all years removing columns with with all missing values atored in inputs/dataset_no_missing.feather file resulting from data_preparation
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7890 entries, 0 to 7889
Data columns (total 40 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Course             7890 non-null   object  
 1   Class              7890 non-null   object  
 2   Sex                7890 non-null   category
 3   1                  7890 non-null   int64   
 4   12                 7890 non-null   int64   
 5   20                 7890 non-null   int64   
 6   10                 7890 non-null   int64   
 7   11                 7890 non-null   int64   
 8   32                 7890 non-null   int64   
 9   38                 7890 non-null   int64   
 10  42                 7890 non-null   int64   
 11  35                 7890 non-null   int64   
 12  50           

In [ ]:
doc.add_heading('compose_datasets', 2)
for key, dataset in compose_datasets.items():
    print(f"Key: {key}")
    print(dataset.columns)

Key: 242
Index(['Course', 'Class', 'Sex', '1', '12', '20', '10', '11', '32', '38', '42',
       '35', '50', '30', '31', '36', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'Bon',
       'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2',
       'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'FinalGradeInteger', 'Total_AP',
       'Total_PP', 'Total_SPA', 'Total_Bon', 'Year', 'FinalGrade'],
      dtype='object')
Key: 6769
Index(['Course', 'Class', 'Sex', '1', '12', '20', '10', '11', '32', '38', '42',
       '35', '50', '30', '31', '36', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'Bon',
       'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2',
       'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'FinalGradeInteger', 'Total_AP',
       'Total_PP', 'Total_SPA', 'Total_Bon', 'Year', '2', '5', '8', '9', '78',
       '123', 'PP3_5'],
      dtype='object')
Key: 6835
Index(['Course', 'Class', 'Sex', '1', '12', '20', '10', '11', '32', '38', '42',
       '35', '50', '30', '31', '36', '34', '72', 'SP

In [ ]:
doc.add_heading('Lists with Activities information', 2)
all_lists_changed = {}
# Update the column names for each dataset and also update the all_lists dictionary
for year, dataset in datasets.items():
    activity_dict = dataset_params[str(year)]['activity_dictionary']
    sync_columns = all_lists[str(year)]['basic_lists']['listSyncColumns']
    datasets[year], column_mapping = update_column_names(dataset, activity_dict, sync_columns)
    all_lists_changed[str(year)] = update_all_lists(all_lists.copy(), str(year), column_mapping)

doc.add_heading('datasets', 2)

for year, dataset in datasets.items():
    print(f"Year: {year}")
    print(dataset.columns)

Year: 2016
Index(['Course', 'Class', 'Sex', 'freq', '1', '3', '24', '12', '20', '22',
       ...
       'Week2_AP', 'Week3_AP', 'Week4_AP', 'Week5_AP', 'Week1_PP', 'Week2_PP',
       'Week3_PP', 'Week4_PP', 'Week5_PP', 'Passed'],
      dtype='object', length=105)
Year: 2017
Index(['Course', 'Class', 'Sex', 'Commuting', 'freq', '1', '3', '24', '12',
       '20',
       ...
       'Week2_AP', 'Week3_AP', 'Week4_AP', 'Week5_AP', 'Week1_PP', 'Week2_PP',
       'Week3_PP', 'Week4_PP', 'Week5_PP', 'Passed'],
      dtype='object', length=107)
Year: 2018
Index(['Course', 'Class', 'Sex', 'Commuting', 'freq', '1', '3', '24', '12',
       '20',
       ...
       'Week2_AP', 'Week3_AP', 'Week4_AP', 'Week5_AP', 'Week1_PP', 'Week2_PP',
       'Week3_PP', 'Week4_PP', 'Week5_PP', 'Passed'],
      dtype='object', length=107)
Year: 2019
Index(['Course', 'Class', 'Sex', 'Commuting', '1', '3', '18', '24', '20', '22',
       ...
       'Week2_AP', 'Week3_AP', 'Week4_AP', 'Week5_AP', 'Week1_PP', 'Week2_PP',

In [ ]:
# Verify the changes in all_lists
for year in all_lists_changed:
    print(f"Year: {year}")
    print(all_lists_changed[year]['basic_lists']['listAllColumns'])

Year: 2016
['Course', 'Class', 'Sex', 'freq', '1', '3', '24', '12', '20', '22', '4', '25', '10', '11', '1_5', '16', '23', '32', '38', '42', '23', '35', '58', '23', '50', '54', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '131', '132', '83', '94', '86', '87', '88', '89', '91', '93', '72', '95', '105', '106', '103', '96', '97', '101', '102', '77', '104', 'SPA1', 'SPA2', 'SPA3', 'Bon', 'BonCoord', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PP4_1', 'PP4_2', 'PP5_1', 'PP5_2', 'PP5_3', 'PTP1', 'PTP2', 'PTP3', 'PTP4', 'PTP5', 'FinalGradeInteger', 'FinalGrade', 'Total_AP', 'Total_PP', 'Total_SPA', 'Total_Bon', 'Week1_AP', 'Week2_AP', 'Week3_AP', 'Week4_AP', 'Week5_AP', 'Week1_PP', 'Week2_PP', 'Week3_PP', 'Week4_PP', 'Week5_PP', 'Passed']
Year: 2017
['Course', 'Class', 'Sex', 'Commuting', 'freq', '1', '3', '24', '12', '20', '22', '4', '25', '10', '11', '16', '23', '32', '38', '42', '23', '35',

In [ ]:
doc.add_heading('Combined dataset made in this notebook', 2)

# Ensure the datasets dictionary is properly indexed and columns are unique
for year, df in datasets.items():
    datasets[year] = df.reset_index(drop=True)
    datasets[year] = ensure_unique_columns(datasets[year])

# Combine all the dataframes from the dictionary 'datasets' into a single dataframe
combined_df = combine_datasets(datasets)

# Check the combined dataframe
print("Combined DataFrame shape:", combined_df.shape)
print("Combined DataFrame columns:", combined_df.columns)


Combined DataFrame shape: (7890, 185)
Combined DataFrame columns: Index(['Course', 'Class', 'Sex', 'freq', '1', '3', '24', '12', '20', '22',
       ...
       'SPA4', 'PP4_4', 'PP4_5', '2', '5', '8', '9', '78', '123', 'PP3_5'],
      dtype='object', length=185)


In [ ]:
doc.add_heading('Verify the loaded datasets', 2)

# Verify the loaded datasets
print("datasets (from data_prepation but with the IDs as column names) keys:", datasets.keys())
print()
print("compose_datasets (from data_preparation notebook) keys:", compose_datasets.keys()) # imported from data_preparation
print()
print("dataset_complete (dataset_no_missing.feather from data_preparation notebook) size = ", len(dataset_complete))
print()
print("-"*50)
print("combined_df dataframe (created in this notebook) size = ", len(combined_df)) # combined in this notebook
print()
print("combined_df dataframe (created in this notebook)  columns: ", combined_df.columns)
print()
print("combined_df dataframe (created in this notebook)  info:")
print(combined_df.info())
print("combined_df dataframe (created in this notebook) statistics:\n", combined_df.describe())

datasets (from data_prepation but with the IDs as column names) keys: dict_keys([2016, 2017, 2018, 2019, 2022, 2023, 2024])

compose_datasets (from data_preparation notebook) keys: dict_keys([242, 6769, 6835, 3453, 5660, 3500, 6711, 1179, 6721, 1055, 3409, 2290, 1121, 6781, 4578, 0, 2230, 5590, 4481, 2300])

dataset_complete (dataset_no_missing.feather from data_preparation notebook) size =  7890

--------------------------------------------------
combined_df dataframe (created in this notebook) size =  7890

combined_df dataframe (created in this notebook)  columns:  Index(['Course', 'Class', 'Sex', 'freq', '1', '3', '24', '12', '20', '22',
       ...
       'SPA4', 'PP4_4', 'PP4_5', '2', '5', '8', '9', '78', '123', 'PP3_5'],
      dtype='object', length=185)

combined_df dataframe (created in this notebook)  info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7890 entries, 0 to 7889
Columns: 185 entries, Course to PP3_5
dtypes: category(1), float64(136), int64(44), object(4)
memo

In [ ]:
print("combined_df dataframe (created in this notebook) missing values:")

check_missing_values(combined_df)

combined_df dataframe (created in this notebook) missing values:
Missing values for Course dataset:
[]
Missing values for Class dataset:
[]
Missing values for Sex dataset:
[]
Missing values for freq dataset:
[1055]
Missing values for 1 dataset:
[]
Missing values for 3 dataset:
[3409]
Missing values for 24 dataset:
[3409]
Missing values for 12 dataset:
[]
Missing values for 20 dataset:
[]
Missing values for 22 dataset:
[3409]
Missing values for 4 dataset:
[3409]
Missing values for 25 dataset:
[1121]
Missing values for 10 dataset:
[]
Missing values for 11 dataset:
[]
Missing values for 1_5 dataset:
[6721]
Missing values for 16 dataset:
[3409]
Missing values for 23 dataset:
[2230]
Missing values for 32 dataset:
[]
Missing values for 38 dataset:
[]
Missing values for 42 dataset:
[]
Missing values for 23_1 dataset:
[3409]
Missing values for 35 dataset:
[]
Missing values for 58 dataset:
[3409]
Missing values for 23_2 dataset:
[4464]
Missing values for 50 dataset:
[]
Missing values for 54 dat

# 4. Application of Auxiliary Methods

## 4.1. Check Missing Values

In [ ]:
# Check for missing values in the datasets for each CTCT edition
print("datasets (from data_prepation but with the IDs as column names) - Checking missing values: ")
check_missing_values(datasets)

datasets (from data_prepation but with the IDs as column names) - Checking missing values: 
Missing values for 2016 dataset:
FinalGrade    63
dtype: int64
Missing values for 2017 dataset:
Series([], dtype: int64)
Missing values for 2018 dataset:
FinalGrade    29
dtype: int64
Missing values for 2019 dataset:
FinalGrade    42
dtype: int64
Missing values for 2022 dataset:
Series([], dtype: int64)
Missing values for 2023 dataset:
FinalGrade    49
dtype: int64
Missing values for 2024 dataset:
FinalGrade    59
dtype: int64


## 4.2. Filter datasets for FinalGrade and FinalGradeInteger Prediction

In [ ]:
# Filter out rows with missing or zero FinalGrade in the datasets
filtered_datasets = filter_missing_and_zero_final_grade(datasets)

for year, dataset in filtered_datasets.items():
    print(f"After filtering, {year} dataset has {len(dataset[dataset['FinalGrade']>0])} positive FinalGrade values.")

After filtering, 2016 dataset has 1106 rows. Before had 1169. Dropped 63 instances/students.
After filtering, 2017 dataset has 1078 rows. Before had 1152. Dropped 74 instances/students.
After filtering, 2018 dataset has 1076 rows. Before had 1105. Dropped 29 instances/students.
After filtering, 2019 dataset has 1013 rows. Before had 1055. Dropped 42 instances/students.
After filtering, 2022 dataset has 1138 rows. Before had 1179. Dropped 41 instances/students.
After filtering, 2023 dataset has 1060 rows. Before had 1109. Dropped 49 instances/students.
After filtering, 2024 dataset has 1062 rows. Before had 1121. Dropped 59 instances/students.
After filtering, 2016 dataset has 1106 positive FinalGrade values.
After filtering, 2017 dataset has 1078 positive FinalGrade values.
After filtering, 2018 dataset has 1076 positive FinalGrade values.
After filtering, 2019 dataset has 1013 positive FinalGrade values.
After filtering, 2022 dataset has 1138 positive FinalGrade values.
After filterin

In [ ]:
# Filter out missing and zero final grades in dataset_complete
print("Filtering missing and zero final grades in dataset_complete...")
filtered_dataset_complete = dataset_complete.dropna(subset=['FinalGradeInteger'])
filtered_dataset_complete = filtered_dataset_complete[filtered_dataset_complete['FinalGradeInteger'] != 0]

# Filter out missing and zero final grades in dataset_complete
print("Filtering missing and zero final grades in combined_df...")
filtered_combined_df = combined_df.dropna(subset=['FinalGradeInteger'])
filtered_combined_df = filtered_combined_df[filtered_combined_df['FinalGradeInteger'] != 0]

Filtering missing and zero final grades in dataset_complete...
Filtering missing and zero final grades in combined_df...


In [ ]:
filtered_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7498 entries, 0 to 7830
Columns: 185 entries, Course to PP3_5
dtypes: category(1), float64(136), int64(44), object(4)
memory usage: 10.6+ MB


In [ ]:
filtered_dataset_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7498 entries, 0 to 7830
Data columns (total 40 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Course             7498 non-null   object  
 1   Class              7498 non-null   object  
 2   Sex                7498 non-null   category
 3   1                  7498 non-null   int64   
 4   12                 7498 non-null   int64   
 5   20                 7498 non-null   int64   
 6   10                 7498 non-null   int64   
 7   11                 7498 non-null   int64   
 8   32                 7498 non-null   int64   
 9   38                 7498 non-null   int64   
 10  42                 7498 non-null   int64   
 11  35                 7498 non-null   int64   
 12  50                 7498 non-null   int64   
 13  30                 7498 non-null   int64   
 14  31                 7498 non-null   int64   
 15  36                 7498 non-null   int64   
 16  34         

# 5. Feature Selection

In [ ]:
doc.add_heading('Output of Feature Selection Report', 0)
all_lists = all_lists_changed

## 5.1. Exclude Columns from Selection

In [ ]:
doc.add_heading('Exclude Columns', 1)

In [ ]:
exclude_columns = {}
fixed_exclude_columns = ['SS', 'Commuting', 'freq', 'Class', 'Course', 'Sex']
for year in dataset_years:
    exclude_columns[str(year)] = set()
    all_lists_year = all_lists[str(year)]
    exclude_columns[str(year)].update(fixed_exclude_columns)
    exclude_columns[str(year)].update(all_lists_year['basic_lists']['listSPAColumns'])
    exclude_columns[str(year)].update(all_lists_year['basic_lists']['listBonColumns'])
    exclude_columns[str(year)].update(all_lists_year['basic_lists']['listGrades'])
    exclude_columns[str(year)].update(all_lists_year['basic_lists']['listCategorical'])
    exclude_columns[str(year)].update(all_lists_year['basic_lists']['listPPointsColumns'])
    exclude_columns[str(year)].update(all_lists_year['created_columns']['TotalColumns'])
    exclude_columns[str(year)].update(all_lists_year['created_columns']['OtherColumns'])
    exclude_columns[str(year)].update(all_lists_year['created_columns']['WeeklyAP'])
    exclude_columns[str(year)].update(all_lists_year['created_columns']['WeeklyPP'])
    exclude_columns[str(year)].update(all_lists_year['weekly_and_participation_lists']['listWeek3'])
    exclude_columns[str(year)].update(all_lists_year['weekly_and_participation_lists']['listWeek4'])
    exclude_columns[str(year)].update(all_lists_year['weekly_and_participation_lists']['listWeek5'])
    exclude_columns[str(year)].update(all_lists_year['weekly_and_participation_lists']['listPPWeek3'])
    exclude_columns[str(year)].update(all_lists_year['weekly_and_participation_lists']['listPPWeek4'])
    exclude_columns[str(year)].update(all_lists_year['weekly_and_participation_lists']['listPPWeek5'])

print("Exclude columns defined successfully.")

Exclude columns defined successfully.


In [ ]:
exclude_columns_less_restricted = {}
fixed_exclude_columns = ['SS', 'Commuting', 'freq', 'Class', 'Course', 'Sex']
for year in dataset_years:
    exclude_columns_less_restricted [str(year)] = set()
    all_lists_year = all_lists[str(year)]
    exclude_columns_less_restricted[str(year)].update(fixed_exclude_columns)
    exclude_columns_less_restricted[str(year)].update(all_lists_year['basic_lists']['listSPAColumns'])
    exclude_columns_less_restricted[str(year)].update(all_lists_year['basic_lists']['listBonColumns'])
    exclude_columns_less_restricted[str(year)].update(all_lists_year['basic_lists']['listGrades'])
    exclude_columns[str(year)].update(all_lists_year['basic_lists']['listCategorical'])
    exclude_columns_less_restricted[str(year)].update(all_lists_year['basic_lists']['listPPointsColumns'])
    exclude_columns_less_restricted[str(year)].update(all_lists_year['created_columns']['TotalColumns'])
    exclude_columns_less_restricted[str(year)].update(all_lists_year['created_columns']['OtherColumns'])
    exclude_columns_less_restricted[str(year)].update(all_lists_year['created_columns']['WeeklyAP'])
    exclude_columns_less_restricted[str(year)].update(all_lists_year['created_columns']['WeeklyPP'])

print("Exclude columns defined successfully.")

Exclude columns defined successfully.


In [ ]:
exclude_columns_not_restricted = {}
fixed_exclude_columns = ['SS', 'Commuting', 'freq', 'Class', 'Course', 'Sex']
for year in dataset_years:
    exclude_columns_not_restricted [str(year)] = set()
    all_lists_year = all_lists[str(year)]
    exclude_columns_not_restricted [str(year)].update(fixed_exclude_columns)

print("Exclude columns defined successfully.")

Exclude columns defined successfully.


In [ ]:
# List of exclusion types and corresponding exclusion column dictionaries
exclusion_columns_dicts = {
    'more_restricted': exclude_columns,
    'restricted': exclude_columns_less_restricted,
    'less_restricted': exclude_columns_not_restricted
}

In [ ]:
# Aggregate columns to exclude from all specified years
years_to_exclude = ['2016', '2017', '2018', '2019', '2022', '2023', '2024']
aggregated_exclude_columns = {}
aggregated_exclude_columns['more_restricted'] = set()
aggregated_exclude_columns['restricted'] = set()
aggregated_exclude_columns['less_restricted'] = set()

for year in years_to_exclude:
    aggregated_exclude_columns['more_restricted'].update(exclude_columns[year])
    aggregated_exclude_columns['restricted'].update(exclude_columns_less_restricted[year])
    aggregated_exclude_columns['less_restricted'].update(exclude_columns_not_restricted[year])

# Convert the set back to a list
aggregated_exclude_columns['more_restricted'] = list(aggregated_exclude_columns['more_restricted'])
aggregated_exclude_columns['restricted'] = list(aggregated_exclude_columns['restricted'])
aggregated_exclude_columns['less_restricted'] = list(aggregated_exclude_columns['less_restricted'])

### 5.1.1. Save combined_df for prediction

In [ ]:
create_and_save_datasets(combined_df, exclude_columns, 'Passed', output_dir)
create_and_save_datasets(filtered_combined_df, exclude_columns, 'FinalGrade', output_dir, filename_suffix="filtered_combined_df")
create_and_save_datasets(combined_df, exclude_columns, 'Total_AP', output_dir)
create_and_save_datasets(combined_df, exclude_columns, 'Total_PP', output_dir)
create_and_save_datasets(combined_df, exclude_columns, 'Total_SPA', output_dir)

Saved selected features to output/combined_df_Passed.feather.
Saved selected features to output/filtered_combined_df_FinalGrade.feather.
Saved selected features to output/combined_df_Total_AP.feather.
Saved selected features to output/combined_df_Total_PP.feather.
Saved selected features to output/combined_df_Total_SPA.feather.


## 5.2. Methods

###### select_by_variance

In [ ]:
def select_by_variance(dataset, exclude_columns, variance_threshold=0.0):
    """
    Select columns from a dataset based on variance threshold and print their variance scores.

    Args:
    - dataset (pd.DataFrame): The input dataset containing the data to be filtered.
    - exclude_columns (list): List of columns to be excluded from the variance threshold calculation.
    - variance_threshold (float): The minimum variance threshold for features to be selected. Default is 0.0.

    Returns:
    - selected_columns (pd.Index): A sorted list of columns by variance in descending order.
    """
    # Select numerical columns, drop excluded columns, and fill missing values with 0
    numerical_cols = dataset.select_dtypes(include=[np.number]).drop(columns=exclude_columns, errors='ignore').fillna(0)

    # Initialize the VarianceThreshold with the specified threshold
    sel = VarianceThreshold(threshold=variance_threshold)

    # Apply the variance threshold to the selected numerical columns
    sel.fit(numerical_cols)
    variances = sel.variances_

    # Get the columns that pass the threshold
    selected_columns = numerical_cols.columns[sel.get_support()]

    # Sort the selected columns by their variance in descending order
    sorted_indices = np.argsort(variances[sel.get_support()])[::-1]  # Sort only the selected variances
    sorted_columns = selected_columns[sorted_indices]

    # Print the sorted columns with their variance scores
    variance_table = pd.DataFrame({'Feature': sorted_columns, 'Variance': variances[sel.get_support()][sorted_indices]})
    print("Variance Table")
    print(variance_table)
    print("-" * 50)
    print("")
    # Return only the sorted columns
    return sorted_columns


###### feature_selection_analysis  (for original datasets)
(for use on the dictionary with a dataframe per CTCT edition)

In [ ]:
def feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT']):
    """
    Perform feature selection analysis using multiple methods.

    Args:
    - datasets (dict): Dictionary of datasets by year.
    - all_lists (dict): Dictionary containing various lists of columns for each dataset year.
    - dataset_params (dict): Dictionary containing dataset parameters.
    - target_column (str): The target column for the analysis.
    - methods (list): List of methods to use for feature selection ('MI', 'RF', 'DT').

    Returns:
    - selected_features (dict): Dictionary of selected features for each year and method.
    """
    def perform_feature_selection_methods(dataset, all_lists, dataset_params, target_column, methods):
        """
        Perform feature selection using specified methods on the given dataset.
        """
        def select_by_mutual_info(features, target, target_is_continuous=False, random_state=2024):
            # Use mutual_info_regression for continuous targets and mutual_info_classif for categorical
            if target_is_continuous:
                mi = mutual_info_regression(features, target, random_state=random_state)
            else:
                mi = mutual_info_classif(features, target, random_state=random_state)
            mi_scores = pd.Series(mi, index=features.columns)
            return mi_scores

        def select_by_random_forest(numerical_cols, target_encoded, is_regression=False, random_state=2024):
            if is_regression:
                rf = RandomForestRegressor(random_state=random_state)
            else:
                rf = RandomForestClassifier(random_state=random_state)
            rf.fit(numerical_cols, target_encoded)
            rf_scores = pd.Series(rf.feature_importances_, index=numerical_cols.columns)
            return rf_scores

        def select_by_decision_tree(numerical_cols, target_encoded, is_regression=False, random_state=2024):
            if is_regression:
                dt = DecisionTreeRegressor(random_state=random_state)
            else:
                dt = DecisionTreeClassifier(random_state=random_state)
            dt.fit(numerical_cols, target_encoded)
            dt_scores = pd.Series(dt.feature_importances_, index=numerical_cols.columns)
            return dt_scores

        selected_features = {}

        # Prepare the data for feature selection
        numerical_cols = dataset.drop(columns=[target_column], errors='ignore').select_dtypes(include=[np.number])

        # Handle missing values
        imputer = SimpleImputer(strategy='constant', fill_value=0)
        numerical_cols = pd.DataFrame(imputer.fit_transform(numerical_cols), columns=numerical_cols.columns)

        # Identify categorical columns
        categorical_cols = [col for col in dataset.columns if col in all_lists.get('basic_lists', {}).get('listCategorical', [])]
        categorical_cols = [col for col in categorical_cols if col not in [target_column]]

        # Ensure proper encoding based on the method used
        encoded_datasets = {}

        if 'MI' in methods:
            # OneHotEncoding for MI
            if categorical_cols:
                enc = OneHotEncoder(sparse_output=False, drop='first')
                encoded_categorical = enc.fit_transform(dataset[categorical_cols].astype(str))
                encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=enc.get_feature_names_out(categorical_cols))
                encoded_datasets['MI'] = numerical_cols.join(encoded_categorical_df)
            else:
                encoded_datasets['MI'] = numerical_cols

        if 'RF' in methods or 'DT' in methods:
            # LabelEncoding for RF and DT
            le = LabelEncoder()
            for col in categorical_cols:
                if col in dataset.columns:
                    dataset[col] = le.fit_transform(dataset[col].astype(str))

            if 'RF' in methods:
                encoded_datasets['RF'] = dataset[numerical_cols.columns.tolist() + categorical_cols]
            if 'DT' in methods:
                encoded_datasets['DT'] = dataset[numerical_cols.columns.tolist() + categorical_cols]

        # Determine if the target is continuous or categorical
        target_is_continuous = pd.api.types.is_numeric_dtype(dataset[target_column])

        # For continuous target (regression), use the target as-is; otherwise, encode it
        target_encoded = dataset[target_column] if target_is_continuous else dataset[target_column]

        for method in methods:
            if method == 'MI' and 'MI' in encoded_datasets:
                mi_scores = select_by_mutual_info(encoded_datasets['MI'], target_encoded, target_is_continuous=target_is_continuous)
                threshold_mi = 0.5 * mi_scores.median()
                important_features_mi = mi_scores[mi_scores > threshold_mi].index.tolist()
                selected_features['MI'] = important_features_mi

            if method == 'RF' and 'RF' in encoded_datasets:
                rf_scores = select_by_random_forest(encoded_datasets['RF'], target_encoded, is_regression=target_is_continuous)
                threshold_rf = 0.5 * rf_scores.median()
                important_features_rf = rf_scores[rf_scores > threshold_rf].index.tolist()
                selected_features['RF'] = important_features_rf

            if method == 'DT' and 'DT' in encoded_datasets:
                dt_scores = select_by_decision_tree(encoded_datasets['DT'], target_encoded, is_regression=target_is_continuous)
                threshold_dt = 0.5 * dt_scores.median()
                important_features_dt = dt_scores[dt_scores > threshold_dt].index.tolist()
                selected_features['DT'] = important_features_dt

        return selected_features

    # Initialize the dictionary to store selected features
    selected_features = {}

    for year, dataset in datasets.items():
        print(f"Feature Selection Analysis for the dataset of year: {year} with target column: {target_column}")

        if target_column not in dataset.columns:
            print(f"Skipping year {year}: Target column not found in dataset.")
            continue

        # Exclude columns and handle categorical variables properly
        columns_to_exclude = list(exclude_columns.get(str(year), []))
        selected_columns = [col for col in dataset.columns if col not in columns_to_exclude + [target_column]]

        if len(selected_columns) == 0:
            print(f"No columns selected for year {year} after applying column exclusion.")
            continue

        # Add the target column back to the selected columns
        selected_columns.append(target_column)

        # Subset the dataset with the selected columns
        dataset_reduced = dataset[selected_columns]

        # Handle missing values
        imputer = SimpleImputer(strategy='constant', fill_value=0)
        dataset_reduced_imputed = pd.DataFrame(imputer.fit_transform(dataset_reduced), columns=dataset_reduced.columns)

        # Process the rest of the function with the reduced and imputed columns
        selected_features[year] = perform_feature_selection_methods(dataset_reduced_imputed, all_lists.get(str(year), {}), dataset_params, target_column, methods)

        print("-" * 50)

    return selected_features

###### feature_selection_for_regression

In [ ]:
def feature_selection_for_regression(datasets, target_column, exclude_columns, methods=['FR', 'RF', 'Lasso']):
    """
    Perform feature selection analysis for regression using multiple methods.

    Args:
    - datasets (dict): Dictionary of datasets by year.
    - target_column (str): The target column for regression.
    - exclude_columns (list): List of columns to exclude from feature selection.
    - methods (list): List of methods to use for feature selection ('FR', 'RF', 'Lasso').

    Returns:
    - selected_features (dict): Dictionary of selected features for each year and method.
    """
    selected_features = {}

    for year, dataset in datasets.items():
        print(f"Feature Selection Analysis for the dataset of year: {year} with target column: {target_column}")

        if target_column not in dataset.columns:
            print(f"Skipping year {year}: Target column not found in dataset.")
            continue

        # Exclude columns
        numerical_cols = dataset.drop(columns=exclude_columns + [target_column], errors='ignore').select_dtypes(include=[np.number])
        target = dataset[target_column]

        # Perform feature selection using the specified methods
        selected_features[year] = {}

        if 'FR' in methods:  # Feature selection using F-regression
            #print("Calculating F-regression scores...")
            selector = SelectKBest(score_func=f_regression, k='all')
            selector.fit(numerical_cols, target)
            fr_scores = pd.Series(selector.scores_, index=numerical_cols.columns)
            threshold_fr = 0.5 * fr_scores.median()
            selected_features[year]['FR'] = fr_scores[fr_scores > threshold_fr].index.tolist()
            #print(f"Selected features by F-regression with threshold {threshold_fr}: {selected_features[year]['FR']}")

        if 'RF' in methods:  # Feature selection using Random Forest
            #print("Calculating Random Forest (RF) feature importances...")
            rf = RandomForestRegressor(random_state=2024)
            rf.fit(numerical_cols, target)
            rf_importances = pd.Series(rf.feature_importances_, index=numerical_cols.columns)
            threshold_rf = 0.5 * rf_importances.median()
            selected_features[year]['RF'] = rf_importances[rf_importances > threshold_rf].index.tolist()
            #print(f"Selected features by Random Forest with threshold {threshold_rf}: {selected_features[year]['RF']}")

        if 'Lasso' in methods:  # Feature selection using Lasso regression
            #print("Calculating Lasso coefficients...")
            lasso = Lasso(alpha=0.01, random_state=2024)
            lasso.fit(numerical_cols, target)
            lasso_coefficients = pd.Series(lasso.coef_, index=numerical_cols.columns)
            selected_features[year]['Lasso'] = lasso_coefficients[lasso_coefficients != 0].index.tolist()
            #print(f"Selected features by Lasso: {selected_features[year]['Lasso']}")

        print("-" * 50)

    return selected_features


###### feature_selection_analysis_generic (for compose_datasets)

In [ ]:
def feature_selection_analysis_generic(datasets, all_lists, target_column, exclude_columns, methods=['MI', 'RF', 'DT']):
    """
    Perform feature selection analysis using multiple methods.

    Args:
    - datasets (dict): Dictionary of datasets by key.
    - all_lists (dict): Dictionary containing various lists of columns for each dataset key.
    - target_column (str): The target column for the analysis.
    - exclude_columns (dict): Dictionary of columns to exclude for each year.
    - methods (list): List of methods to use for feature selection ('MI', 'RF', 'DT').

    Returns:
    - selected_features (dict): Dictionary of selected features for each key and method.
    """
    def select_by_mutual_info(features, target, target_is_continuous=False, random_state=2024):
        #print("Calculating Mutual Information (MI) scores...")
        if target_is_continuous:
            mi = mutual_info_regression(features, target, random_state=random_state)
        else:
            mi = mutual_info_classif(features, target, random_state=random_state)
        mi_scores = pd.Series(mi, index=features.columns)
        #print("MI scores calculated:", mi_scores.head())
        return mi_scores

    def select_by_random_forest(numerical_cols, target_encoded, random_state=2024):
        #print("Calculating Random Forest (RF) feature importances...")
        rf = RandomForestClassifier(random_state=random_state)
        rf.fit(numerical_cols, target_encoded)
        rf_scores = pd.Series(rf.feature_importances_, index=numerical_cols.columns)
        #print("RF importances calculated:", rf_scores.head())
        return rf_scores

    def select_by_decision_tree(numerical_cols, target_encoded, random_state=2024):
        #print("Calculating Decision Tree (DT) feature importances...")
        dt = DecisionTreeClassifier(random_state=random_state)
        dt.fit(numerical_cols, target_encoded)
        dt_scores = pd.Series(dt.feature_importances_, index=numerical_cols.columns)
        #print("DT importances calculated:", dt_scores.head())
        return dt_scores

    selected_features = {}

    for key, dataset in datasets.items():
        print(f"Feature Selection Analysis for the dataset with key: {key} with target column: {target_column}")

        if target_column not in dataset.columns:
            print(f"Skipping key {key}: Target column not found in dataset.")
            continue

        # Usar todos os anos presentes no dataset
        years = dataset['Year'].unique()

        # Ensure columns are excluded correctly
        columns_to_exclude = set(exclude_columns.get(str(years[0]), [])) | set([target_column])
        numerical_cols = dataset.select_dtypes(include=[np.number]).drop(columns=columns_to_exclude, errors='ignore')

        # Handle missing values
        imputer = SimpleImputer(strategy='constant', fill_value=0)
        numerical_cols = pd.DataFrame(imputer.fit_transform(numerical_cols), columns=numerical_cols.columns)

        selected_features[key] = {}

        for year in years:
            year = str(year)

            # Identify categorical columns for the specific year
            if year in all_lists:
                categorical_cols = [col for col in dataset.columns if col in all_lists[year]['basic_lists']['listCategorical']]
            else:
                print(f"Year {year} not found in all_lists for key {key}. Skipping this year.")
                continue

            categorical_cols = [col for col in categorical_cols if col not in columns_to_exclude]  # Exclude categorical columns that should be excluded

            # Create a copy of the dataset to avoid modifying the original
            dataset_copy = dataset.copy()

            # Apply appropriate encoding for each method
            encoded_datasets = {}

            # OneHotEncoding for MI
            if 'MI' in methods:
                if categorical_cols:
                    #print("Performing OneHotEncoding for MI...")
                    enc = OneHotEncoder(sparse_output=False, drop='first')
                    encoded_categorical = enc.fit_transform(dataset[categorical_cols].astype(str))
                    encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=enc.get_feature_names_out(categorical_cols))
                    encoded_datasets['MI'] = numerical_cols.join(encoded_categorical_df)
                    #print(f"Encoded MI dataset: {encoded_datasets['MI'].shape}")
                else:
                    encoded_datasets['MI'] = numerical_cols  # If no categorical columns, MI can use numerical data directly

            # LabelEncoding for RF and DT
            if 'RF' in methods or 'DT' in methods:
                #print("Performing LabelEncoding for RF and DT...")
                le = LabelEncoder()
                for col in categorical_cols:
                    if col in dataset_copy.columns:
                        dataset_copy[col] = le.fit_transform(dataset_copy[col].astype(str))

                if 'RF' in methods:
                    encoded_datasets['RF'] = dataset_copy[numerical_cols.columns.tolist() + categorical_cols]
                    #print(f"Encoded RF dataset: {encoded_datasets['RF'].shape}")

                if 'DT' in methods:
                    encoded_datasets['DT'] = dataset_copy[numerical_cols.columns.tolist() + categorical_cols]
                    #print(f"Encoded DT dataset: {encoded_datasets['DT'].shape}")

            # Handle target column discretization or encoding
            if pd.api.types.is_numeric_dtype(dataset[target_column]):
                target_encoded = dataset[target_column]
            else:
                target_encoded = dataset[target_column]

            for method in methods:
                if method == 'MI' and 'MI' in encoded_datasets:
                    mi_scores = select_by_mutual_info(encoded_datasets['MI'], target_encoded)
                    threshold_mi = 0.5 * mi_scores.median()
                    important_features_mi = mi_scores[mi_scores > threshold_mi].index.tolist()
                    selected_features[key][f'MI_{year}'] = important_features_mi
                    #print(f"Selected features by MI with threshold {threshold_mi}: {important_features_mi}")

                if method == 'RF' and 'RF' in encoded_datasets:
                    rf_scores = select_by_random_forest(encoded_datasets['RF'], target_encoded)
                    threshold_rf = 0.5 * rf_scores.median()
                    important_features_rf = rf_scores[rf_scores > threshold_rf].index.tolist()
                    selected_features[key][f'RF_{year}'] = important_features_rf
                    #print(f"Selected features by RF with threshold {threshold_rf}: {important_features_rf}")

                if method == 'DT' and 'DT' in encoded_datasets:
                    dt_scores = select_by_decision_tree(encoded_datasets['DT'], target_encoded)
                    threshold_dt = 0.5 * dt_scores.median()
                    important_features_dt = dt_scores[dt_scores > threshold_dt].index.tolist()
                    selected_features[key][f'DT_{year}'] = important_features_dt
                    #print(f"Selected features by DT with threshold {threshold_dt}: {important_features_dt}")

        print("-" * 50)

    return selected_features


###### extract_common_features
###### extract_common_features_continuous

In [ ]:
def extract_common_features(selected_features):
    common_features = {'MI': [], 'RF': [], 'DT': []}
    methods = common_features.keys()

    for method in methods:
        # Get the list of features for each year for the current method
        features_per_year = [set(features[method]) for year, features in selected_features.items() if method in features]

        # Find the intersection of all feature sets
        if features_per_year:
            common_features[method] = list(set.intersection(*features_per_year))

    return common_features

def extract_common_features_continuous(selected_features_cont):
    common_features = {'FR': [], 'RF': [], 'Lasso': []}
    methods = common_features.keys()

    for method in methods:
        # Get the list of features for each year for the current method
        features_per_year = [set(features[method]) for year, features in selected_features_cont.items() if method in features]

        # Find the intersection of all feature sets
        if features_per_year:
            common_features[method] = list(set.intersection(*features_per_year))

    return common_features


In [ ]:
def extract_common_features_per_compose_dataset(selected_features, compose_datasets):
    common_features_per_dataset = {}

    for dataset_key, compose_df in compose_datasets.items():
        # Extract the years available in the current compose dataset
        dataset_years = compose_df['Year'].unique()
        common_features = {'MI': [], 'RF': [], 'DT': []}

        for method in common_features.keys():
            # Collect features for the available years and the current method
            features_per_year = []
            for year in dataset_years:
                key = f"{method}_{year}"
                if key in selected_features.get(dataset_key, {}):
                    features_per_year.append(set(selected_features[dataset_key][key]))

            # Find the intersection of all feature sets for the current method
            if features_per_year:
                common_features[method] = list(set.intersection(*features_per_year))
                if not common_features[method]:  # Check if the intersection is empty
                    print(f"No common features found for {dataset_key} (method {method})")

        common_features_per_dataset[dataset_key] = common_features

    return common_features_per_dataset

def extract_common_features_continuous_per_compose_dataset(selected_features_cont, compose_datasets):
    common_features_per_dataset = {}

    for dataset_key, compose_df in compose_datasets.items():
        # Extract the years available in the current compose dataset
        dataset_years = compose_df['Year'].unique()
        common_features = {'FR': [], 'RF': [], 'Lasso': []}

        for method in common_features.keys():
            # Collect features for the available years and the current method
            features_per_year = []
            for year in dataset_years:
                key = f"{method}_{year}"
                if key in selected_features_cont.get(dataset_key, {}):
                    features_per_year.append(set(selected_features_cont[dataset_key][key]))

            # Find the intersection of all feature sets for the current method
            if features_per_year:
                common_features[method] = list(set.intersection(*features_per_year))
                if not common_features[method]:  # Check if the intersection is empty
                    print(f"No common features found for {dataset_key} (method {method})")

        common_features_per_dataset[dataset_key] = common_features

    return common_features_per_dataset


## 5.3. Application of Methods for Selection

In [ ]:
doc.add_heading('Variance Threshold 0.05', 1)

In [ ]:
# Applying the modified function in the loop
high_variance_features = {}
variance_threshold = 0.05

for year, dataset in datasets.items():
    print(f"Applying variance filter for dataset of year: {year}")

    # Call the select_by_variance method and get only the selected columns
    high_variance_features[str(year)] = select_by_variance(
        dataset,
        exclude_columns_less_restricted[str(year)],
        variance_threshold
    )

    print(f"Columns retained from {year} dataset based on variance threshold = {variance_threshold}:")
    print(high_variance_features[str(year)].tolist())

    missing_week1 = set(all_lists[str(year)]['weekly_and_participation_lists']['listWeek1']) - set(high_variance_features[str(year)])
    missing_week2 = set(all_lists[str(year)]['weekly_and_participation_lists']['listWeek2']) - set(high_variance_features[str(year)])
    print("-" * 50)
    print()
    print("----------------------------------------------------------------------")

print("-" * 50)
print()
print("----------------------------------------------------------------------")
print("Converted:")
converted_features = {}
for year in high_variance_features:
    converted_features[year] = convert_feature_ids_to_activity_names(high_variance_features[year], dataset_params)
    print(f"Converted features for year {year}: {converted_features[year]}")

Applying variance filter for dataset of year: 2016
Variance Table
   Feature  Variance
0       91  2.477985
1      129  2.251002
2      128  2.247608
3       30  2.100181
4      130  2.000701
5      127  1.871316
6      103  1.760783
7      102  1.604145
8       77  1.483133
9       88  1.437878
10      58  1.436170
11     101  1.429324
12      93  1.399207
13      97  1.397496
14    23_1  1.365687
15      11  1.257817
16      96  1.196216
17      72  1.082637
18      89  1.043503
19      87  1.041490
20    23_2  1.013609
21      86  1.010377
22      22  0.971728
23      31  0.923272
24      32  0.891707
25     125  0.874281
26      10  0.856720
27      34  0.826952
28      35  0.784875
29      38  0.573078
30      16  0.520366
31     106  0.303547
32     132  0.244888
33      54  0.234488
34     104  0.178995
35     131  0.176698
36      83  0.172945
37     105  0.166648
38      50  0.156476
39      53  0.145166
40      95  0.141816
41      94  0.130860
42      24  0.129082
43      20

In [ ]:
get_activity_name_by_id(117, dataset_params, 2022)

'Purchase of computer equipment (Solver)'

In [ ]:
get_activity_name_by_id(76, dataset_params, 2023)

'Communication using audiovisual media - Slides'

### 5.3.1. For predicting if a student pass or fail
Feature selection on datasets dictionary of dataframes, one per each year (preprocessed original dataframes).

In [ ]:
doc.add_heading('Predicting Passed', 2)

In [ ]:
exclusion_type = 'more_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Passed'

selected_features_predict_passed_more_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_predict_passed_more_restricted, datasets, target_column, output_dir, "predict_passed", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Passed
--------------------------------------------------


In [ ]:
doc.add_heading('More restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_predict_passed_more_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_predict_passed_more_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '3', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34'], 'RF': ['24', '12', '20', '22', '25', '10', '11', '23', '32', '38', '42', '23_1', '35', '50', '30', '51', '31', '28', '53', '36', '55', '56', '34'], 'DT': ['12', '20', '25', '10', '11', '23', '32', '42', '23_1', '23_2', '50', '30', '51', '31', '53', '34']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '94_1'], 'RF': ['1', '12', '20', '25', '10', '23', '32', '38', '42', '35', '58', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '94_1'], 'DT': ['1', '20', '10', '11', '38', '23_1', '35', '58', '23_2', '50', '30', '31', '52', '28', '56', '94_1']}, 2018: {'MI': ['1', '24', '12', '20', '25', '10', '11', '23', '18', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '55', '56', '34', '94_1'], 'RF': [

In [ ]:
print(extract_common_features(selected_features_predict_passed_more_restricted))

{'MI': ['38', '31', '34', '42', '10', '35', '30', '50', '20', '12', '32'], 'RF': ['31', '10', '35', '12', '30'], 'DT': []}


In [ ]:
exclusion_type = 'restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Passed'

selected_features_predict_passed_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_predict_passed_restricted, datasets, target_column, output_dir, "predict_passed", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Passed
--------------------------------------------------


In [ ]:
doc.add_heading('Restricted results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_predict_passed_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_predict_passed_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '131', '94', '86', '87', '89', '91', '72', '95', '105', '106', '103', '96', '104'], 'RF': ['24', '12', '20', '22', '25', '16', '32', '42', '35', '50', '30', '31', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '83', '94', '86', '87', '89', '91', '72', '95', '105', '103', '96', '97', '101', '104'], 'DT': ['25', '34', '127', '86', '72', '95', '103', '97']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '94', '94_1', '4_2', '86', '87', '89', '91', '72', '95', '104', '96', '103'], 'RF': ['3', '24', '20', '10', '23', '32', '38', '35', '23_2', '30', '31', '53', '36', '55', '56', '34', '124', 

In [ ]:
print(extract_common_features(selected_features_predict_passed_restricted))

{'MI': ['38', '31', '34', '72', '10', '30', '50', '20', '12', '32', '36'], 'RF': ['72'], 'DT': []}


In [ ]:
exclusion_type = 'less_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Passed'

selected_features_predict_passed_less_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_predict_passed_less_restricted, datasets, target_column, output_dir, "predict_passed", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Passed
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Passed
--------------------------------------------------


In [ ]:
doc.add_heading('Less restricted results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_predict_passed_less_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_predict_passed_less_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '94', '86', '87', '89', '91', '72', '95', '105', '103', '96', '104', 'SPA1', 'SPA2', 'SPA3', 'BonCoord', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PP4_1', 'PP4_2', 'PP5_1', 'PP5_2', 'PP5_3', 'PTP1', 'PTP2', 'PTP3', 'PTP4', 'PTP5', 'FinalGradeInteger', 'FinalGrade', 'Total_AP', 'Total_PP', 'Total_SPA', 'Week1_AP', 'Week2_AP', 'Week3_AP', 'Week4_AP', 'Week5_AP', 'Week1_PP', 'Week2_PP', 'Week3_PP', 'Week4_PP', 'Week5_PP'], 'RF': ['FinalGradeInteger', 'FinalGrade'], 'DT': ['FinalGrade']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '94', '94_1', '4_2', '86', '87', '91', '72

In [ ]:
print(extract_common_features(selected_features_predict_passed_less_restricted))

{'MI': ['Week1_AP', 'SPA2', '35', 'Total_PP', 'SPA3', 'Week3_PP', 'FinalGrade', '42', '72', 'Week1_PP', 'PP3_1', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', 'Week3_AP', '31', 'Week2_PP', 'PP2_3', 'Week2_AP', '10', 'PTP1', 'SPA1', '20', '32', '38', 'PP1_3', '34', 'PP2_2', '12', '30'], 'RF': ['FinalGradeInteger', 'FinalGrade'], 'DT': []}


### 5.3.2. For predicting *FinalGrade* from students that passed
Feature selection on datasets dictionary of dataframes, one per each year (preprocessed original dataframes).

In [ ]:
doc.add_heading('Predicting FinalGrade from students that passed', 2)

In [ ]:
print("More Restricted Results")
print("----------------------------------------------------------------------")
selected_features_finalgrade_more_restricted = feature_selection_for_regression(
    filtered_datasets, # using only students that passed CTCT
    target_column='FinalGrade',
    exclude_columns=aggregated_exclude_columns['more_restricted'],
    methods=['FR', 'RF', 'Lasso']
)

print("Restricted Results")
print("----------------------------------------------------------------------")
selected_features_finalgrade_restricted = feature_selection_for_regression(
    filtered_datasets, # using only students that passed CTCT
    target_column='FinalGrade',
    exclude_columns=aggregated_exclude_columns['restricted'],
    methods=['FR', 'RF', 'Lasso']
)

print("Less Restricted Results")
print("----------------------------------------------------------------------")
selected_features_finalgrade_less_restricted = feature_selection_for_regression(
    filtered_datasets, # using only students that passed CTCT
    target_column='FinalGrade',
    exclude_columns=aggregated_exclude_columns['less_restricted'],
    methods=['FR', 'RF', 'Lasso']
)

More Restricted Results
----------------------------------------------------------------------
Feature Selection Analysis for the dataset of year: 2016 with target column: FinalGrade
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: FinalGrade
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: FinalGrade
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: FinalGrade
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: FinalGrade
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: FinalGrade
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target co

In [ ]:
print("Selected features for predicting FinalGrade:")
print("------------------ More restricted set")
print(selected_features_finalgrade_more_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_finalgrade_more_restricted, dataset_params)
print(converted_features)
print("------------------ Restricted set")
print(selected_features_finalgrade_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_finalgrade_restricted, dataset_params)
print(converted_features)
print("------------------ Less restricted set")
print(selected_features_finalgrade_less_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_finalgrade_less_restricted, dataset_params)
print(converted_features)

Selected features for predicting FinalGrade:
------------------ More restricted set
{2016: {'FR': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34'], 'RF': ['1', '24', '12', '20', '22', '25', '10', '11', '16', '23', '32', '38', '23_1', '35', '58', '23_2', '50', '30', '31', '52', '28', '53', '34'], 'Lasso': ['1', '3', '24', '22', '4', '25', '10', '11', '16', '23', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '54', '30', '31', '52', '28', '53', '36', '56', '34']}, 2017: {'FR': ['1', '24', '12', '20', '25', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '94_1'], 'RF': ['24', '12', '20', '22', '25', '10', '11', '23', '32', '38', '23_1', '35', '58', '23_2', '50', '30', '31', '52', '36', '55', '56', '34', '94_1', '72_1'], 'Lasso': ['24', '12', '20', '22', '25', '10', '11', '16', '23', '32', '38', '23_1', '35', '58', '23_2', '50', '54', '30', '31', '52', '53', '36

In [ ]:
print(extract_common_features_continuous(selected_features_finalgrade_more_restricted))
print(extract_common_features_continuous(selected_features_finalgrade_restricted))
print(extract_common_features_continuous(selected_features_finalgrade_less_restricted))

{'FR': ['34', '31', '42', '35', '30', '32', '36'], 'RF': ['38', '34', '31', '11', '35', '10', '30', '50', '32'], 'Lasso': ['38', '34', '31', '10', '11', '30', '50', '32']}
{'FR': ['31', '34', '42', '72', '35', '30', '32', '36'], 'RF': ['38', '31', '34', '10', '35', '30', '50', '32'], 'Lasso': ['38', '31', '34', '32', '11', '35', '10', '50', '30']}
{'FR': ['Week1_AP', 'SPA2', 'Total_PP', 'SPA3', 'Week3_PP', '72', 'PP3_1', 'PP2_1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', 'Week3_AP', '31', 'Week2_PP', 'Week2_AP', 'SPA1', '32', 'Total_SPA', '34', 'PP2_2', '30'], 'RF': ['PTP2', 'Bon', 'SPA2', 'Week1_AP', '11', '35', 'Total_PP', 'SPA3', 'Week3_PP', 'Week1_PP', 'PP3_1', 'PP1_2', 'Total_AP', 'FinalGradeInteger', 'Week3_AP', '31', 'Week2_PP', 'Week2_AP', '10', 'SPA1', '32', '38', 'PTP3', 'Total_SPA', '34', '30'], 'Lasso': ['FinalGradeInteger', 'Week3_AP', 'Week1_AP', 'Week2_AP', 'Total_PP']}


### 5.3.3. For predicting *Total_AP*
Feature selection on datasets dictionary of dataframes, one per each year (preprocessed original dataframes).

In [ ]:
print("More Restricted Results")
print("----------------------------------------------------------------------")

# For Total_AP
target_column = 'Total_AP'
selected_features_total_ap_regression_more_restricted = feature_selection_for_regression(
    datasets,
    target_column=target_column,
    exclude_columns=aggregated_exclude_columns['more_restricted'],
    methods=['FR', 'RF', 'Lasso']
)

# For Total_PP
target_column = 'Total_PP'
selected_features_total_pp_regression_more_restricted = feature_selection_for_regression(
    datasets,
    target_column=target_column,
    exclude_columns=aggregated_exclude_columns['more_restricted'],
    methods=['FR', 'RF', 'Lasso']
)

# For Total_SPA
target_column = 'Total_SPA'
selected_features_total_spa_regression_more_restricted = feature_selection_for_regression(
    datasets,
    target_column=target_column,
    exclude_columns=aggregated_exclude_columns['more_restricted'],
    methods=['FR', 'RF', 'Lasso']
)


More Restricted Results
----------------------------------------------------------------------
Feature Selection Analysis for the dataset of year: 2016 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_

In [ ]:
# Combine features across all years for each method
combined_features_total_ap_regression_more_restricted = combine_selected_features(selected_features_total_ap_regression_more_restricted)
combined_features_total_pp_regression_more_restricted = combine_selected_features(selected_features_total_pp_regression_more_restricted)
combined_features_total_spa_regression_more_restricted = combine_selected_features(selected_features_total_spa_regression_more_restricted)


In [ ]:
print("Selected features for predicting Total_AP:")
print(selected_features_total_ap_regression_more_restricted)
print("Combined features for predicting Total_AP:")
print(combined_features_total_ap_regression_more_restricted)


Selected features for predicting Total_AP:
{2016: {'FR': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34'], 'RF': ['1', '24', '12', '20', '22', '25', '10', '11', '16', '23', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '30', '31', '52', '28', '53', '36', '55', '34'], 'Lasso': ['1', '3', '24', '12', '20', '22', '4', '25', '10', '11', '16', '23', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '54', '30', '51', '31', '52', '28', '53', '36', '34']}, 2017: {'FR': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34'], 'RF': ['12', '20', '25', '10', '11', '23', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '94_1', '72_1'], 'Lasso': ['1', '3', '24', '12', '20', '22', '4', '25', '10', '11', '16', '23', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '54', '30', '51', '

In [ ]:
doc.add_heading('Predicting Total_AP', 2)

In [ ]:
exclusion_type = 'more_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Total_AP'

selected_features_total_ap_more_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_total_ap_more_restricted, datasets, target_column, output_dir, "predict_total_ap", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_AP
--------------------------------------------------


In [ ]:
doc.add_heading('More restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_total_ap_more_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_total_ap_more_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34'], 'RF': ['1', '24', '12', '20', '22', '25', '10', '11', '16', '23', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '30', '31', '52', '28', '53', '36', '55', '34'], 'DT': ['1', '24', '12', '20', '22', '25', '10', '11', '23', '32', '38', '23_1', '35', '58', '23_2', '50', '30', '31', '28', '53', '34']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34'], 'RF': ['12', '20', '25', '10', '11', '23', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '94_1', '72_1'], 'DT': ['12', '20', '22', '25', '10', '11', '16', '32', '38', '23_1', '35', '58', '23_2', '50', '30', '31', '52', '56', '34', '94_1', '72_1']}, 2018: {'MI': ['1', '24', '12', '20', '25', '10', '23', '18', '32', '

In [ ]:
print(extract_common_features(selected_features_total_ap_more_restricted))

{'MI': ['38', '31', '34', '42', '10', '35', '30', '50', '20', '12', '32', '1', '36'], 'RF': ['38', '31', '34', '32', '11', '35', '10', '12', '30'], 'DT': ['38', '32', '34', '30']}


In [ ]:
exclusion_type = 'restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Total_AP'

selected_features_total_ap_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_total_ap_restricted, datasets, target_column, output_dir, "predict_total_ap", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_AP
--------------------------------------------------


In [ ]:
doc.add_heading('Restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_total_ap_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_total_ap_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '94', '86', '87', '89', '91', '72', '95', '105', '103', '96', '104'], 'RF': ['24', '20', '25', '10', '11', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '30', '51', '31', '52', '28', '53', '55', '34', '124', '125', '127', '128', '129', '130', '131', '83', '94', '86', '87', '89', '91', '72', '95', '105', '103', '96', '97', '101', '102', '77', '104'], 'DT': ['12', '25', '10', '11', '16', '38', '23_1', '35', '58', '23_2', '30', '51', '31', '52', '55', '34', '124', '125', '127', '128', '129', '130', '131', '83', '94', '86', '87', '88', '89', '91', '93', '72', '105', '106', '103', '96', '97', '101', '102', '77', '104']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34'

In [ ]:
print(extract_common_features(selected_features_total_ap_restricted))

{'MI': ['38', '31', '34', '42', '72', '35', '10', '30', '50', '20', '12', '32', '1', '36'], 'RF': ['38', '31', '34', '35', '30', '32'], 'DT': ['30', '10', '38']}


In [ ]:
exclusion_type = 'less_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Total_AP'

selected_features_total_ap_less_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_total_ap_less_restricted, datasets, target_column, output_dir, "predict_total_ap", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_AP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_AP
--------------------------------------------------


In [ ]:
doc.add_heading('Less restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_total_ap_less_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_total_ap_less_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '94', '86', '87', '89', '91', '72', '95', '105', '103', '96', '104', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PP4_1', 'PP4_2', 'PP5_1', 'PP5_2', 'PP5_3', 'PTP1', 'PTP2', 'PTP3', 'PTP4', 'PTP5', 'FinalGradeInteger', 'FinalGrade', 'Total_PP', 'Total_SPA', 'Week1_AP', 'Week2_AP', 'Week3_AP', 'Week4_AP', 'Week5_AP', 'Week1_PP', 'Week2_PP', 'Week3_PP', 'Week4_PP', 'Week5_PP', 'Passed'], 'RF': ['12', '20', '22', '10', '11', '23', '32', '38', '23_1', '35', '58', '54', '30', '31', '34', '125', '127', '128', '129', '130', '131', '83', '86', '87', '88', '89', '91', '93', '72', '95', '103', '96', '97', '101', '102', '77', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP2_1', 'PP2_3', 'PP3_2', 'PP4_1', 'PP5_1', 'P

In [ ]:
print(extract_common_features(selected_features_total_ap_less_restricted))

{'MI': ['Week1_AP', 'SPA2', '35', 'Total_PP', 'SPA3', 'FinalGrade', 'Week3_PP', '42', '72', 'Week1_PP', 'PP3_1', '50', 'PP3_2', 'PP2_1', 'Passed', 'PP3_3', '1', '36', 'FinalGradeInteger', 'Week3_AP', '31', 'Week2_PP', 'Week2_AP', 'SPA1', '32', '38', 'Total_SPA', '34', 'PP2_2', '12', '30'], 'RF': ['FinalGradeInteger', '38', 'Week3_AP', 'Total_SPA', 'FinalGrade', 'Week3_PP', 'Week1_AP', 'SPA2', 'PP2_3', 'Week2_PP', 'Week2_AP', '35', '30', 'Total_PP', 'SPA3', '32'], 'DT': ['38', 'PTP3', 'Week3_AP', 'Total_SPA', 'FinalGrade', 'Week3_PP', 'Week1_AP', '31', 'Week2_AP', '11', 'Week1_PP', '30', 'Total_PP', 'SPA1', '32']}


### 5.3.4. For predicting *Total_PP*
Feature selection on datasets dictionary of dataframes, one per each year (preprocessed original dataframes).

In [ ]:
doc.add_heading('Predicting Total_PP', 2)

In [ ]:
exclusion_type = 'more_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Total_PP'

selected_features_total_pp_more_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_total_pp_more_restricted, datasets, target_column, output_dir, "predict_total_pp", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_PP
--------------------------------------------------


In [ ]:
doc.add_heading('More restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_total_pp_more_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_total_pp_more_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34'], 'RF': ['1', '3', '24', '20', '22', '25', '10', '11', '16', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '30', '51', '31', '28', '53', '36', '55', '56', '34'], 'DT': ['1', '3', '24', '12', '20', '22', '10', '11', '16', '32', '38', '23_1', '35', '58', '23_2', '50', '30', '31', '28', '53', '34']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '94_1'], 'RF': ['24', '12', '20', '22', '25', '10', '11', '23', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '30', '31', '52', '28', '53', '36', '55', '56', '34', '94_1', '72_1'], 'DT': ['24', '12', '22', '25', '10', '11', '16', '23', '32', '38', '23_1', '35', '58', '23_2', '50', '54', '30', '31', '52', '36', '55', '34', '94_1', '72_1']}, 2018: {'MI': ['1', '24', '12', '20', '

In [ ]:
print(extract_common_features(selected_features_total_pp_more_restricted))

{'MI': ['38', '31', '34', '42', '32', '10', '35', '50', '12', '30', '1', '36'], 'RF': ['38', '34', '31', '11', '35', '10', '30', '32'], 'DT': ['38', '34', '31', '32', '11', '35', '10', '30']}


In [ ]:
exclusion_type = 'restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Total_PP'

selected_features_total_pp_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_total_pp_restricted, datasets, target_column, output_dir, "predict_total_pp", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_PP
--------------------------------------------------


In [ ]:
doc.add_heading('Restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_total_pp_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_total_pp_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '94', '86', '87', '89', '91', '72', '95', '105', '103', '96', '104'], 'RF': ['1', '3', '24', '12', '20', '22', '4', '10', '11', '16', '23', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '30', '31', '52', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '86', '87', '88', '89', '91', '93', '72', '95', '106', '103', '96', '97', '101', '102', '77', '104'], 'DT': ['1', '3', '24', '12', '20', '22', '4', '10', '11', '16', '32', '38', '42', '23_1', '58', '23_2', '50', '30', '31', '52', '28', '53', '34', '124', '125', '127', '128', '129', '130', '131', '86', '88', '89', '91', '93', '72', '95', '106', '103', '96', '97', '101', '102', '77', '104']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31'

In [ ]:
print(extract_common_features(selected_features_total_pp_restricted))

{'MI': ['38', '31', '34', '42', '72', '35', '10', '30', '50', '20', '12', '32', '1', '36'], 'RF': ['38', '31', '34', '11', '35', '10', '30', '32'], 'DT': ['38', '31', '34', '11', '10', '30', '32', '1']}


In [ ]:
exclusion_type = 'less_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Total_PP'

selected_features_total_pp_less_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_total_pp_less_restricted, datasets, target_column, output_dir, "predict_total_pp", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_PP
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_PP
--------------------------------------------------


In [ ]:
doc.add_heading('Less restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_total_pp_less_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_total_pp_less_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '94', '86', '87', '89', '91', '72', '95', '105', '103', '96', '104', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PP4_1', 'PP4_2', 'PP5_1', 'PP5_2', 'PP5_3', 'PTP1', 'PTP2', 'PTP3', 'PTP4', 'PTP5', 'FinalGradeInteger', 'FinalGrade', 'Total_AP', 'Total_SPA', 'Week1_AP', 'Week2_AP', 'Week3_AP', 'Week4_AP', 'Week5_AP', 'Week1_PP', 'Week2_PP', 'Week3_PP', 'Week4_PP', 'Week5_PP', 'Passed'], 'RF': ['22', '10', '11', '32', '38', '23_1', '35', '58', '23_2', '50', '30', '31', '28', '53', '34', '125', '126', '127', '128', '129', '130', '131', '83', '86', '87', '88', '89', '91', '93', '72', '95', '106', '103', '96', '97', '101', '102', '77', '104', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP

In [ ]:
print(extract_common_features(selected_features_total_pp_less_restricted))

{'MI': ['PTP2', 'Week1_AP', 'SPA2', '35', 'SPA3', 'PP1_1', 'Week3_PP', 'FinalGrade', '42', '72', 'Week1_PP', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'Passed', 'PP2_1', '1', '36', 'PP3_3', 'Total_AP', 'FinalGradeInteger', 'Week3_AP', '31', 'Week2_PP', 'PP2_3', 'Week2_AP', '10', 'PTP1', 'SPA1', '20', '32', '38', 'PTP3', 'PP1_3', 'Total_SPA', 'PP2_2', '12', '30'], 'RF': ['PTP2', 'Week1_AP', 'SPA2', '11', '35', 'SPA3', 'PP1_1', 'FinalGrade', 'Week3_PP', 'Week1_PP', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', 'Week3_AP', '31', 'PP2_3', 'Week2_PP', 'Week2_AP', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', 'PP2_2', '30'], 'DT': ['PTP2', 'Total_AP', 'PTP3', 'Week3_AP', 'PP1_3', 'Week3_PP', 'FinalGrade', 'Week1_AP', 'Week2_PP', 'Total_SPA', 'Week2_AP', 'Week1_PP', 'SPA3', 'PP1_2', '30']}


### 5.3.5. For predicting *Total_SPA*
Feature selection on datasets dictionary of dataframes, one per each year (preprocessed original dataframes).

In [ ]:
doc.add_heading('Predicting Total_SPA', 2)

In [ ]:
exclusion_type = 'more_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Total_SPA'

selected_features_total_spa_more_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_total_spa_more_restricted, datasets, target_column, output_dir, "predict_total_spa", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_SPA
--------------------------------------------------


In [ ]:
doc.add_heading('More restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_total_spa_more_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_total_spa_more_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34'], 'RF': ['1', '24', '12', '20', '22', '25', '10', '11', '32', '38', '23_1', '35', '58', '23_2', '50', '30', '51', '31', '28', '53', '36', '55', '56', '34'], 'DT': ['1', '24', '12', '22', '25', '10', '11', '23', '32', '38', '23_1', '35', '58', '23_2', '50', '30', '51', '31', '52', '28', '53', '56']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '94_1'], 'RF': ['12', '20', '22', '25', '10', '11', '32', '38', '42', '23_1', '35', '58', '23_2', '50', '30', '31', '52', '53', '55', '56', '34', '94_1', '72_1'], 'DT': ['24', '12', '22', '25', '10', '11', '16', '23', '32', '38', '23_1', '35', '58', '23_2', '50', '54', '30', '31', '28', '53', '56', '34', '94_1', '72_1']}, 2018: {'MI': ['1', '24', '12', '22', '10', '11', '23', '18', '32', '38',

In [ ]:
print(extract_common_features(selected_features_total_spa_more_restricted))

{'MI': ['34', '31', '42', '10', '35', '30', '12', '32', '1', '36'], 'RF': ['38', '31', '34', '32', '11', '35', '10', '30'], 'DT': ['38', '31', '32', '11', '35', '10', '30']}


In [ ]:
exclusion_type = 'restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Total_SPA'

selected_features_total_spa_restricted = feature_selection_analysis(datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_total_spa_restricted, datasets, target_column, output_dir, "predict_total_spa", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_SPA
--------------------------------------------------


In [ ]:
doc.add_heading('Restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_total_spa_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_total_spa_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '83', '94', '86', '87', '89', '91', '72', '95', '105', '103', '96', '97', '102', '77', '104'], 'RF': ['1', '24', '22', '25', '10', '11', '32', '38', '23_1', '35', '50', '30', '51', '31', '28', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '94', '86', '87', '88', '89', '91', '93', '72', '103', '96', '97', '101', '102', '77', '104'], 'DT': ['1', '24', '12', '22', '25', '10', '11', '16', '32', '38', '23_1', '35', '30', '31', '28', '55', '34', '124', '125', '126', '127', '128', '129', '130', '94', '86', '87', '88', '89', '91', '93', '72', '105', '106', '103', '96', '97', '101', '102', '77']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '58', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '12

In [ ]:
print(extract_common_features(selected_features_total_spa_restricted))

{'MI': ['38', '31', '34', '42', '72', '35', '30', '50', '12', '32'], 'RF': ['38', '31', '34', '10', '11', '35', '30', '32'], 'DT': ['10', '11', '31']}


In [ ]:
selected_features_total_spa_less_restricted = feature_selection_analysis(
    datasets, all_lists, dataset_params, target_column, exclude_columns, methods=['MI', 'RF', 'DT']
)

#save_selected_features_to_feather(selected_features_total_spa_less_restricted, datasets, target_column, output_dir, "predict_total_spa", exclusion_type)

Feature Selection Analysis for the dataset of year: 2016 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2017 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2018 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2019 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2022 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2023 with target column: Total_SPA
--------------------------------------------------
Feature Selection Analysis for the dataset of year: 2024 with target column: Total_SPA
--------------------------------------------------


In [ ]:
doc.add_heading('Less restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_total_spa_less_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names(selected_features_total_spa_less_restricted, dataset_params)
print(converted_features)

Selected Features:
{2016: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '126', '127', '128', '129', '130', '83', '94', '86', '87', '89', '91', '72', '95', '105', '103', '96', '97', '102', '77', '104'], 'RF': ['1', '24', '22', '25', '10', '11', '32', '38', '23_1', '35', '50', '30', '51', '31', '28', '34', '124', '125', '126', '127', '128', '129', '130', '131', '83', '94', '86', '87', '88', '89', '91', '93', '72', '103', '96', '97', '101', '102', '77', '104'], 'DT': ['1', '24', '12', '22', '25', '10', '11', '16', '32', '38', '23_1', '35', '30', '31', '28', '55', '34', '124', '125', '126', '127', '128', '129', '130', '94', '86', '87', '88', '89', '91', '93', '72', '105', '106', '103', '96', '97', '101', '102', '77']}, 2017: {'MI': ['1', '24', '12', '20', '25', '10', '23', '32', '38', '42', '35', '58', '50', '30', '51', '31', '52', '28', '53', '36', '55', '56', '34', '124', '125', '12

In [ ]:
print(extract_common_features(selected_features_total_spa_less_restricted))

{'MI': ['38', '31', '34', '42', '72', '35', '30', '50', '12', '32'], 'RF': ['38', '31', '34', '10', '11', '35', '30', '32'], 'DT': ['10', '11', '31']}


### 5.3.6 Using compose_datasets
Feature selection on datasets with multiple years generated in data_preparation.ipynb file.

In [ ]:
doc.add_heading('Using compose_datasets', 1)

In [ ]:
for key, value in compose_datasets.items():
    print(f"Key: {key}, Dataset Size: {len(value)}")
    print(f"Years in this dataset: {value['Year'].unique()}")

Key: 242, Dataset Size: 7648
Years in this dataset: ['2016' '2017' '2018' '2019' '2022' '2023' '2024']
Key: 6769, Dataset Size: 1121
Years in this dataset: ['2024']
Key: 6835, Dataset Size: 1055
Years in this dataset: ['2019']
Key: 3453, Dataset Size: 4437
Years in this dataset: ['2017' '2019' '2023' '2024']
Key: 5660, Dataset Size: 2230
Years in this dataset: ['2023' '2024']
Key: 3500, Dataset Size: 4390
Years in this dataset: ['2018' '2019' '2023' '2024']
Key: 6711, Dataset Size: 1179
Years in this dataset: ['2022']
Key: 1179, Dataset Size: 6711
Years in this dataset: ['2016' '2017' '2018' '2019' '2023' '2024']
Key: 6721, Dataset Size: 1169
Years in this dataset: ['2016']
Key: 1055, Dataset Size: 6835
Years in this dataset: ['2016' '2017' '2018' '2022' '2023' '2024']
Key: 3409, Dataset Size: 4481
Years in this dataset: ['2016' '2017' '2018' '2019']
Key: 2290, Dataset Size: 5600
Years in this dataset: ['2017' '2018' '2019' '2022' '2023']
Key: 1121, Dataset Size: 6769
Years in this dat

#### For predicting Sex column

In [ ]:
doc.add_heading('For predicting Sex', 2)

In [ ]:
exclusion_type = 'more_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Sex'

selected_features_sex_more_restricted = feature_selection_analysis_generic(compose_datasets, all_lists_id, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_sex_more_restricted, compose_datasets, target_column, output_dir, "predict_sex", exclusion_type)

Feature Selection Analysis for the dataset with key: 242 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6769 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6835 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3453 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 5660 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3500 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6711 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 1179 with target 

In [ ]:
doc.add_heading('More restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_sex_more_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names_complete(selected_features_sex_more_restricted, dataset_params)
print(converted_features)

Selected Features:
{242: {'MI_2016': ['1', '12', '11', '32', '38', '42', '35', '50', '31', '36', '34'], 'RF_2016': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2016': ['10', '11', '32', '38', '35', '50', '30', '31', '34'], 'MI_2017': ['1', '12', '11', '32', '38', '42', '35', '50', '31', '36', '34'], 'RF_2017': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2017': ['10', '11', '32', '38', '35', '50', '30', '31', '34'], 'MI_2018': ['1', '12', '11', '32', '38', '42', '35', '50', '31', '36', '34'], 'RF_2018': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2018': ['10', '11', '32', '38', '35', '50', '30', '31', '34'], 'MI_2019': ['1', '12', '11', '32', '38', '42', '35', '50', '31', '36', '34'], 'RF_2019': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2019': ['10', '11', '32', '38', '35', '50', '30', '31', '34'], 'MI_2022': ['1', '12', '11', '32', '38', '42', '35', '50', '31', '36', '34'], 'RF_2022': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2022': 

In [ ]:
common_features_per_dataset = extract_common_features_per_compose_dataset(selected_features_sex_more_restricted, compose_datasets)
print(common_features_per_dataset)

{242: {'MI': ['38', '34', '31', '42', '11', '35', '50', '12', '32', '1', '36'], 'RF': ['38', '34', '31', '32', '11', '35', '10', '30'], 'DT': ['38', '34', '31', '32', '11', '35', '10', '50', '30']}, 6769: {'MI': ['31', '34', '10', '20', '9', '12', '32', '1', '36'], 'RF': ['31', '11', '10', '35', '5', '20', '9', '32', '38', '2', '34', '42', '8', '50', '30', '1'], 'DT': ['38', '2', '31', '34', '42', '11', '10', '35', '30', '5', '8', '9', '32', '1']}, 6835: {'MI': ['38', '31', '42', '10', '20', '12', '32', '1_8'], 'RF': ['38', '31', '34', '11', '35', '10', '30', '50', '32'], 'DT': ['38', '1_8', '31', '34', '11', '10', '35', '30', '20', '50', '32', '1']}, 3453: {'MI': ['38', '34', '31', '35', '20', '12', '1', '36'], 'RF': ['38', '34', '31', '32', '11', '35', '10', '50', '30'], 'DT': ['38', '34', '31', '32', '11', '35', '10', '50', '30']}, 5660: {'MI': ['7', '41', '10', '11', '35', '20', '50', '32', '1'], 'RF': ['38', '34', '41', '31', '32', '11', '35', '10', '30'], 'DT': ['38', '34', '41',

In [ ]:
exclusion_type = 'restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Sex'

selected_features_sex_restricted = feature_selection_analysis_generic(compose_datasets, all_lists_id, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_sex_restricted, compose_datasets, target_column, output_dir, "predict_sex", exclusion_type)

Feature Selection Analysis for the dataset with key: 242 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6769 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6835 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3453 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 5660 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3500 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6711 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 1179 with target 

In [ ]:
doc.add_heading('Restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_sex_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names_complete(selected_features_sex_restricted, dataset_params)
print(converted_features)

Selected Features:
{242: {'MI_2016': ['1', '12', '20', '32', '38', '35', '30', '31', '36', '72'], 'RF_2016': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72'], 'DT_2016': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72'], 'MI_2017': ['1', '12', '20', '32', '38', '35', '30', '31', '36', '72'], 'RF_2017': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72'], 'DT_2017': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72'], 'MI_2018': ['1', '12', '20', '32', '38', '35', '30', '31', '36', '72'], 'RF_2018': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72'], 'DT_2018': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72'], 'MI_2019': ['1', '12', '20', '32', '38', '35', '30', '31', '36', '72'], 'RF_2019': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72'], 'DT_2019': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72'], 'MI_2022': ['1', '12', '20', '32', '38', '35', '30', '31', '36', '72'], 'RF_2022': ['10', '11', '32', 

In [ ]:
common_features_per_dataset = extract_common_features_per_compose_dataset(selected_features_sex_restricted, compose_datasets)
print(common_features_per_dataset)

{242: {'MI': ['38', '31', '32', '72', '35', '20', '12', '30', '1', '36'], 'RF': ['38', '34', '31', '32', '11', '35', '10', '72', '50', '30'], 'DT': ['38', '34', '31', '32', '11', '35', '10', '72', '50', '30']}, 6769: {'MI': ['9', '12', '30', '32', '72', '123'], 'RF': ['31', '11', '35', '10', '123', '20', '9', '32', '38', '2', '34', '42', '72', '8', '78', '30', '50', '5', '1'], 'DT': ['31', '11', '10', '35', '5', '123', '20', '9', '32', '38', '2', '34', '42', '78', '50', '30']}, 6835: {'MI': ['31', '10', '35', '72', '20', '50', '12', '32', '1'], 'RF': ['38', '31', '34', '11', '35', '10', '30', '72', '50', '32'], 'DT': ['38', '31', '34', '11', '35', '10', '30', '72', '50', '32', '36']}, 3453: {'MI': ['34', '31', '42', '10', '35', '72', '20', '32', '1', '36'], 'RF': ['38', '34', '31', '32', '11', '35', '10', '72', '50', '30'], 'DT': ['38', '34', '31', '32', '11', '10', '35', '72', '20', '50', '30']}, 5660: {'MI': ['60', '34', '31', '42', '11', '72', '74', '82', '20', '12', '30', '1', '61'

In [ ]:
exclusion_type = 'less_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Sex'

selected_features_sex_less_restricted = feature_selection_analysis_generic(compose_datasets, all_lists_id, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_sex_less_restricted, compose_datasets, target_column, output_dir, "predict_sex", exclusion_type)

Feature Selection Analysis for the dataset with key: 242 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6769 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6835 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3453 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 5660 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3500 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6711 with target column: Sex
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 1179 with target 

In [ ]:
doc.add_heading('Less restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_sex_less_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names_complete(selected_features_sex_less_restricted, dataset_params)
print(converted_features)

Selected Features:
{242: {'MI_2016': ['1', '20', '42', '35', '30', '31', '36', 'SPA2', 'SPA3', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP2', 'FinalGradeInteger', 'Total_AP', 'Total_PP', 'Total_SPA', 'FinalGrade'], 'RF_2016': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'FinalGradeInteger', 'Total_AP', 'Total_PP', 'Total_SPA', 'FinalGrade'], 'DT_2016': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'FinalGradeInteger', 'Total_AP', 'Total_PP', 'Total_SPA', 'FinalGrade'], 'MI_2017': ['1', '20', '42', '35', '30', '31', '36', 'SPA2', 'SPA3', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP2', 'FinalGradeInteger', 'Total_AP', 'Total_PP', 'Total_SPA', 'FinalGrade'], 

In [ ]:
common_features_per_dataset = extract_common_features_per_compose_dataset(selected_features_sex_less_restricted, compose_datasets)
print(common_features_per_dataset)

{242: {'MI': ['PTP2', 'SPA2', '35', 'Total_PP', 'SPA3', 'FinalGrade', '42', 'PP3_1', 'PP3_2', 'PP1_2', 'PP2_1', '1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '20', 'PP1_3', 'Total_SPA', '30'], 'RF': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30'], 'DT': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30']}, 6769: {'MI': ['PTP2', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', 'SPA2', '123', 'PTP1', 'PP3_5', '38', '2', 'Total_SPA', 'Total_Bon', '72', '8', 'PP2_2', '50', 'PP3_2', 'PP2_1', '1', '30', 'PP3_3', '36'], 'RF': ['PTP2', 'SPA2', '11', '35', '

#### For predicting FinalGradeInteger column

In [ ]:
print("Number of Students had negative to CTCT discipline:")
for key, value in compose_datasets.items():
    print(f"Key: {key}, Categories of 'Year' column:\n{value['Year'].value_counts()}")
    print(f"Negatives: {len(value[value['FinalGradeInteger']==0])}")
    print("-" * 50  )

Number of Students had negative to CTCT discipline:
Key: 242, Categories of 'Year' column:
Year
2022    1179
2017    1152
2016    1106
2018    1076
2024    1062
2023    1060
2019    1013
Name: count, dtype: int64
Negatives: 150
--------------------------------------------------
Key: 6769, Categories of 'Year' column:
Year
2024    1121
Name: count, dtype: int64
Negatives: 59
--------------------------------------------------
Key: 6835, Categories of 'Year' column:
Year
2019    1055
Name: count, dtype: int64
Negatives: 42
--------------------------------------------------
Key: 3453, Categories of 'Year' column:
Year
2017    1152
2024    1121
2023    1109
2019    1055
Name: count, dtype: int64
Negatives: 232
--------------------------------------------------
Key: 5660, Categories of 'Year' column:
Year
2024    1121
2023    1109
Name: count, dtype: int64
Negatives: 108
--------------------------------------------------
Key: 3500, Categories of 'Year' column:
Year
2024    1121
2023    1109


In [ ]:
doc.add_heading('For predicting FinalGradeInteger', 2)

In [ ]:
exclusion_type = 'more_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'FinalGradeInteger'

selected_features_finalgradeinteger_more_restricted = feature_selection_analysis_generic(compose_datasets, all_lists_id, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_finalgradeinteger_more_restricted, compose_datasets, target_column, output_dir, "predict_finalgradeinteger", exclusion_type)

Feature Selection Analysis for the dataset with key: 242 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6769 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6835 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3453 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 5660 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3500 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6711 with target column: FinalGradeInteger
-----------------------

In [ ]:
doc.add_heading('More restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_finalgradeinteger_more_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names_complete(selected_features_finalgradeinteger_more_restricted, dataset_params)
print(converted_features)

Selected Features:
{242: {'MI_2016': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34'], 'RF_2016': ['20', '10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2016': ['10', '11', '32', '38', '35', '30', '31', '34'], 'MI_2017': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34'], 'RF_2017': ['20', '10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2017': ['10', '11', '32', '38', '35', '30', '31', '34'], 'MI_2018': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34'], 'RF_2018': ['20', '10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2018': ['10', '11', '32', '38', '35', '30', '31', '34'], 'MI_2019': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34'], 'RF_2019': ['20', '10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2019': ['10', '11', '32', '38', '35', '30', '31', '34'], 'MI_2022': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34'], 'RF_2022': [

In [ ]:
common_features_per_dataset = extract_common_features_per_compose_dataset(selected_features_sex_less_restricted, compose_datasets)
print(common_features_per_dataset)

{242: {'MI': ['PTP2', 'SPA2', '35', 'Total_PP', 'SPA3', 'FinalGrade', '42', 'PP3_1', 'PP3_2', 'PP1_2', 'PP2_1', '1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '20', 'PP1_3', 'Total_SPA', '30'], 'RF': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30'], 'DT': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30']}, 6769: {'MI': ['PTP2', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', 'SPA2', '123', 'PTP1', 'PP3_5', '38', '2', 'Total_SPA', 'Total_Bon', '72', '8', 'PP2_2', '50', 'PP3_2', 'PP2_1', '1', '30', 'PP3_3', '36'], 'RF': ['PTP2', 'SPA2', '11', '35', '

In [ ]:
exclusion_type = 'restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'FinalGradeInteger'

selected_features_finalgradeinteger_restricted = feature_selection_analysis_generic(compose_datasets, all_lists_id, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_finalgradeinteger_restricted, compose_datasets, target_column, output_dir, "predict_finalgradeinteger", exclusion_type)

Feature Selection Analysis for the dataset with key: 242 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6769 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6835 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3453 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 5660 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3500 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6711 with target column: FinalGradeInteger
-----------------------

In [ ]:
doc.add_heading('Restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_finalgradeinteger_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names_complete(selected_features_finalgradeinteger_restricted, dataset_params)
print(converted_features)

Selected Features:
{242: {'MI_2016': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34', '72'], 'RF_2016': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'DT_2016': ['20', '10', '11', '32', '38', '35', '30', '31', '34', '72'], 'MI_2017': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34', '72'], 'RF_2017': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'DT_2017': ['20', '10', '11', '32', '38', '35', '30', '31', '34', '72'], 'MI_2018': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34', '72'], 'RF_2018': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'DT_2018': ['20', '10', '11', '32', '38', '35', '30', '31', '34', '72'], 'MI_2019': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34', '72'], 'RF_2019': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'DT_2019': ['20', '10', '11', '32', '38', '35', '30', '31', '34', '72'], 'MI_2022': ['1', '12', '20', '

In [ ]:
common_features_per_dataset = extract_common_features_per_compose_dataset(selected_features_sex_less_restricted, compose_datasets)
print(common_features_per_dataset)

{242: {'MI': ['PTP2', 'SPA2', '35', 'Total_PP', 'SPA3', 'FinalGrade', '42', 'PP3_1', 'PP3_2', 'PP1_2', 'PP2_1', '1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '20', 'PP1_3', 'Total_SPA', '30'], 'RF': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30'], 'DT': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30']}, 6769: {'MI': ['PTP2', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', 'SPA2', '123', 'PTP1', 'PP3_5', '38', '2', 'Total_SPA', 'Total_Bon', '72', '8', 'PP2_2', '50', 'PP3_2', 'PP2_1', '1', '30', 'PP3_3', '36'], 'RF': ['PTP2', 'SPA2', '11', '35', '

In [ ]:
exclusion_type = 'less_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'FinalGradeInteger'

selected_features_finalgradeinteger_less_restricted = feature_selection_analysis_generic(compose_datasets, all_lists_id, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_finalgradeinteger_less_restricted, compose_datasets, target_column, output_dir, "predict_finalgradeinteger", exclusion_type)

Feature Selection Analysis for the dataset with key: 242 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6769 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6835 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3453 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 5660 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3500 with target column: FinalGradeInteger
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6711 with target column: FinalGradeInteger
-----------------------

In [ ]:
doc.add_heading('Less restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_finalgradeinteger_less_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names_complete(selected_features_finalgradeinteger_less_restricted, dataset_params)
print(converted_features)

Selected Features:
{242: {'MI_2016': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'Total_AP', 'Total_PP', 'Total_SPA', 'FinalGrade'], 'RF_2016': ['1', '10', '11', '32', '38', '35', '50', '30', '31', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'Bon', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'Total_AP', 'Total_PP', 'Total_SPA', 'Total_Bon', 'FinalGrade'], 'DT_2016': ['38', '50', '36', '34', 'Bon', 'PTP2', 'PTP3', 'Total_AP', 'Total_PP', 'Total_SPA', 'Total_Bon', 'FinalGrade'], 'MI_2017': ['1', '12', '20', '10', '32', '38', '42', '35', '50', '30', '31', '36', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'Total_AP', 'Total_PP', 'Total_SPA', 'FinalGrade'], 'RF_2017': ['1', '10

In [ ]:
common_features_per_dataset = extract_common_features_per_compose_dataset(selected_features_sex_less_restricted, compose_datasets)
print(common_features_per_dataset)

{242: {'MI': ['PTP2', 'SPA2', '35', 'Total_PP', 'SPA3', 'FinalGrade', '42', 'PP3_1', 'PP3_2', 'PP1_2', 'PP2_1', '1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '20', 'PP1_3', 'Total_SPA', '30'], 'RF': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30'], 'DT': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30']}, 6769: {'MI': ['PTP2', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', 'SPA2', '123', 'PTP1', 'PP3_5', '38', '2', 'Total_SPA', 'Total_Bon', '72', '8', 'PP2_2', '50', 'PP3_2', 'PP2_1', '1', '30', 'PP3_3', '36'], 'RF': ['PTP2', 'SPA2', '11', '35', '

#### For predicting Course column

In [ ]:
doc.add_heading('For predicting Course', 2)

In [ ]:
exclusion_type = 'more_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Course'

selected_features_course_more_restricted = feature_selection_analysis_generic(compose_datasets, all_lists_id, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_course_more_restricted, compose_datasets, target_column, output_dir, "predict_course", exclusion_type)

Feature Selection Analysis for the dataset with key: 242 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6769 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6835 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3453 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 5660 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3500 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6711 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with k

In [ ]:
doc.add_heading('More restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_course_more_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names_complete(selected_features_course_more_restricted, dataset_params)
print(converted_features)

Selected Features:
{242: {'MI_2016': ['1', '12', '20', '11', '38', '35', '30', '31', '34'], 'RF_2016': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2016': ['10', '11', '32', '38', '35', '30', '31', '34'], 'MI_2017': ['1', '12', '20', '11', '38', '35', '30', '31', '34'], 'RF_2017': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2017': ['10', '11', '32', '38', '35', '30', '31', '34'], 'MI_2018': ['1', '12', '20', '11', '38', '35', '30', '31', '34'], 'RF_2018': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2018': ['10', '11', '32', '38', '35', '30', '31', '34'], 'MI_2019': ['1', '12', '20', '11', '38', '35', '30', '31', '34'], 'RF_2019': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2019': ['10', '11', '32', '38', '35', '30', '31', '34'], 'MI_2022': ['1', '12', '20', '11', '38', '35', '30', '31', '34'], 'RF_2022': ['10', '11', '32', '38', '35', '30', '31', '34'], 'DT_2022': ['10', '11', '32', '38', '35', '30', '31', '34'], 'MI_2023': ['1', '12', '20', '11',

In [ ]:
common_features_per_dataset = extract_common_features_per_compose_dataset(selected_features_sex_less_restricted, compose_datasets)
print(common_features_per_dataset)

{242: {'MI': ['PTP2', 'SPA2', '35', 'Total_PP', 'SPA3', 'FinalGrade', '42', 'PP3_1', 'PP3_2', 'PP1_2', 'PP2_1', '1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '20', 'PP1_3', 'Total_SPA', '30'], 'RF': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30'], 'DT': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30']}, 6769: {'MI': ['PTP2', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', 'SPA2', '123', 'PTP1', 'PP3_5', '38', '2', 'Total_SPA', 'Total_Bon', '72', '8', 'PP2_2', '50', 'PP3_2', 'PP2_1', '1', '30', 'PP3_3', '36'], 'RF': ['PTP2', 'SPA2', '11', '35', '

In [ ]:
exclusion_type = 'restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Course'

selected_features_course_restricted = feature_selection_analysis_generic(compose_datasets, all_lists_id, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_course_restricted, compose_datasets, target_column, output_dir, "predict_course", exclusion_type)

Feature Selection Analysis for the dataset with key: 242 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6769 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6835 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3453 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 5660 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3500 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6711 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with k

In [ ]:
doc.add_heading('Restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_course_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names_complete(selected_features_course_restricted, dataset_params)
print(converted_features)

Selected Features:
{242: {'MI_2016': ['1', '20', '11', '32', '38', '35', '30', '36', '72'], 'RF_2016': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'DT_2016': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'MI_2017': ['1', '20', '11', '32', '38', '35', '30', '36', '72'], 'RF_2017': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'DT_2017': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'MI_2018': ['1', '20', '11', '32', '38', '35', '30', '36', '72'], 'RF_2018': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'DT_2018': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'MI_2019': ['1', '20', '11', '32', '38', '35', '30', '36', '72'], 'RF_2019': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'DT_2019': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'MI_2022': ['1', '20', '11', '32', '38', '35', '30', '36', '72'], 'RF_2022': ['10', '11', '32', '38', '35', '30', '31', '34', '72'], 'DT_2022': ['10', '11', '32', '38', '35',

In [ ]:
common_features_per_dataset = extract_common_features_per_compose_dataset(selected_features_sex_less_restricted, compose_datasets)
print(common_features_per_dataset)

{242: {'MI': ['PTP2', 'SPA2', '35', 'Total_PP', 'SPA3', 'FinalGrade', '42', 'PP3_1', 'PP3_2', 'PP1_2', 'PP2_1', '1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '20', 'PP1_3', 'Total_SPA', '30'], 'RF': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30'], 'DT': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30']}, 6769: {'MI': ['PTP2', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', 'SPA2', '123', 'PTP1', 'PP3_5', '38', '2', 'Total_SPA', 'Total_Bon', '72', '8', 'PP2_2', '50', 'PP3_2', 'PP2_1', '1', '30', 'PP3_3', '36'], 'RF': ['PTP2', 'SPA2', '11', '35', '

In [ ]:
exclusion_type = 'less_restricted'
exclude_columns = exclusion_columns_dicts[exclusion_type]
target_column = 'Course'

selected_features_course_less_restricted = feature_selection_analysis_generic(compose_datasets, all_lists_id, target_column, exclude_columns, methods=['MI', 'RF', 'DT'])
#save_selected_features_to_feather(selected_features_course_less_restricted, compose_datasets, target_column, output_dir, "predict_course", exclusion_type)

Feature Selection Analysis for the dataset with key: 242 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6769 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6835 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3453 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 5660 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 3500 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with key: 6711 with target column: Course
--------------------------------------------------
Feature Selection Analysis for the dataset with k

In [ ]:
doc.add_heading('Less restricted Results', 3)

In [ ]:
print("Selected Features:")
print(selected_features_course_less_restricted)
print("Converted:")
converted_features = convert_feature_ids_to_activity_names_complete(selected_features_course_less_restricted, dataset_params)
print(converted_features)

Selected Features:
{242: {'MI_2016': ['1', '20', '32', '38', '42', 'SPA1', 'SPA2', 'SPA3', 'PP1_2', 'PP1_3', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP1', 'PTP2', 'FinalGradeInteger', 'Total_AP', 'Total_PP', 'Total_SPA', 'Total_Bon', 'FinalGrade'], 'RF_2016': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'FinalGradeInteger', 'Total_AP', 'Total_PP', 'Total_SPA', 'FinalGrade'], 'DT_2016': ['10', '11', '32', '38', '35', '50', '30', '31', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'FinalGradeInteger', 'Total_AP', 'Total_PP', 'Total_SPA', 'FinalGrade'], 'MI_2017': ['1', '20', '32', '38', '42', 'SPA1', 'SPA2', 'SPA3', 'PP1_2', 'PP1_3', 'PP2_3', 'PP3_1', 'PP3_2', 'PP3_3', 'PTP1', 'PTP2', 'FinalGradeInteger', 'Total_AP', 'Total_PP', 'Total_SPA', 'Total_Bon',

In [ ]:
common_features_per_dataset = extract_common_features_per_compose_dataset(selected_features_sex_less_restricted, compose_datasets)
print(common_features_per_dataset)

{242: {'MI': ['PTP2', 'SPA2', '35', 'Total_PP', 'SPA3', 'FinalGrade', '42', 'PP3_1', 'PP3_2', 'PP1_2', 'PP2_1', '1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '20', 'PP1_3', 'Total_SPA', '30'], 'RF': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30'], 'DT': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30']}, 6769: {'MI': ['PTP2', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', 'SPA2', '123', 'PTP1', 'PP3_5', '38', '2', 'Total_SPA', 'Total_Bon', '72', '8', 'PP2_2', '50', 'PP3_2', 'PP2_1', '1', '30', 'PP3_3', '36'], 'RF': ['PTP2', 'SPA2', '11', '35', '

In [ ]:
print(common_features_per_dataset)

{242: {'MI': ['PTP2', 'SPA2', '35', 'Total_PP', 'SPA3', 'FinalGrade', '42', 'PP3_1', 'PP3_2', 'PP1_2', 'PP2_1', '1', 'PP3_3', '36', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '20', 'PP1_3', 'Total_SPA', '30'], 'RF': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30'], 'DT': ['PTP2', 'SPA2', '11', '35', 'Total_PP', 'SPA3', 'PP1_1', 'FinalGrade', '72', 'PP3_1', '50', 'PP3_2', 'PP1_2', 'PP2_1', 'PP3_3', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', '10', 'PTP1', 'SPA1', '32', '38', 'PP1_3', 'PTP3', 'Total_SPA', '34', 'PP2_2', '30']}, 6769: {'MI': ['PTP2', 'Total_AP', 'FinalGradeInteger', '31', 'PP2_3', 'SPA2', '123', 'PTP1', 'PP3_5', '38', '2', 'Total_SPA', 'Total_Bon', '72', '8', 'PP2_2', '50', 'PP3_2', 'PP2_1', '1', '30', 'PP3_3', '36'], 'RF': ['PTP2', 'SPA2', '11', '35', '

## 5.4. Save Outputs from Feature Selection

In [ ]:
doc.add_heading('Feature Selection Outputs', 2)

In [ ]:
# Combine features across all years for each method
combined_features_finalgrade_more_restricted = combine_selected_features(selected_features_finalgrade_more_restricted)
combined_features_finalgrade_restricted = combine_selected_features(selected_features_finalgrade_restricted)
combined_features_finalgrade_less_restricted = combine_selected_features(selected_features_finalgrade_less_restricted)

combined_features_passed_more_restricted = combine_selected_features(selected_features_predict_passed_more_restricted)
combined_features_passed_restricted = combine_selected_features(selected_features_predict_passed_restricted)
combined_features_passed_less_restricted = combine_selected_features(selected_features_predict_passed_less_restricted)

combined_features_total_ap_more_restricted = combine_selected_features(selected_features_total_ap_more_restricted)
combined_features_total_ap_restricted = combine_selected_features(selected_features_total_ap_restricted)
combined_features_total_ap_less_restricted = combine_selected_features(selected_features_total_ap_less_restricted)

combined_features_total_pp_more_restricted = combine_selected_features(selected_features_total_pp_more_restricted)
combined_features_total_pp_restricted = combine_selected_features(selected_features_total_pp_restricted)
combined_features_total_pp_less_restricted = combine_selected_features(selected_features_total_pp_less_restricted)

combined_features_total_spa_more_restricted = combine_selected_features(selected_features_total_spa_more_restricted)
combined_features_total_spa_restricted = combine_selected_features(selected_features_total_spa_restricted)
combined_features_total_spa_less_restricted = combine_selected_features(selected_features_total_spa_less_restricted)

# Save the combined features
output_dir = 'output/feather_files'  # Replace with your actual output directory

# Save for FinalGrade
filename_suffix = 'predict_finalgrade'
save_combined_features_to_feather(combined_features_finalgrade_more_restricted, combined_df, 'FinalGrade', output_dir, filename_suffix, 'more_restricted')
save_combined_features_to_feather(combined_features_finalgrade_restricted, combined_df, 'FinalGrade', output_dir, filename_suffix, 'restricted')
save_combined_features_to_feather(combined_features_finalgrade_less_restricted, combined_df, 'FinalGrade', output_dir, filename_suffix, 'less_restricted')

# Save for Passed
filename_suffix = 'predict_passed'
save_combined_features_to_feather(combined_features_passed_more_restricted, combined_df, 'Passed', output_dir, filename_suffix, 'more_restricted')
save_combined_features_to_feather(combined_features_passed_restricted, combined_df, 'Passed', output_dir, filename_suffix, 'restricted')
save_combined_features_to_feather(combined_features_passed_less_restricted, combined_df, 'Passed', output_dir, filename_suffix, 'less_restricted')

# Save for Total_AP
filename_suffix = 'predict_total_ap'
save_combined_features_to_feather(combined_features_total_ap_more_restricted, combined_df, 'Total_AP', output_dir, filename_suffix, 'more_restricted')
save_combined_features_to_feather(combined_features_total_ap_regression_more_restricted, combined_df, 'Total_AP', output_dir, filename_suffix, 'more_restricted_regression')
save_combined_features_to_feather(combined_features_total_ap_restricted, combined_df, 'Total_AP', output_dir, filename_suffix, 'restricted')
save_combined_features_to_feather(combined_features_total_ap_less_restricted, combined_df, 'Total_AP', output_dir, filename_suffix, 'less_restricted')

# Save for Total_PP
filename_suffix = 'predict_total_pp'
save_combined_features_to_feather(combined_features_total_pp_more_restricted, combined_df, 'Total_PP', output_dir, filename_suffix, 'more_restricted')
save_combined_features_to_feather(combined_features_total_pp_regression_more_restricted, combined_df, 'Total_PP', output_dir, filename_suffix, 'more_restricted_regression')
save_combined_features_to_feather(combined_features_total_pp_restricted, combined_df, 'Total_PP', output_dir, filename_suffix, 'restricted')
save_combined_features_to_feather(combined_features_total_pp_less_restricted, combined_df, 'Total_PP', output_dir, filename_suffix, 'less_restricted')

# Save for Total_SPA
filename_suffix = 'predict_total_spa'
save_combined_features_to_feather(combined_features_total_spa_more_restricted, combined_df, 'Total_SPA', output_dir, filename_suffix, 'more_restricted')
save_combined_features_to_feather(combined_features_total_spa_regression_more_restricted, combined_df, 'Total_SPA', output_dir, filename_suffix, 'more_restricted_regression')
save_combined_features_to_feather(combined_features_total_spa_restricted, combined_df, 'Total_SPA', output_dir, filename_suffix, 'restricted')
save_combined_features_to_feather(combined_features_total_spa_less_restricted, combined_df, 'Total_SPA', output_dir, filename_suffix, 'less_restricted')


Saving DataFrame for method 'FR' with exclusion type 'more_restricted':
- Number of features: 49
- Shape of DataFrame: (7890, 50)
- First few columns: ['23', '37', '133', '41', '35']...
Success: Saved output/feather_files/predict_finalgrade_FR_more_restricted.feather with FinalGrade as the last column.
Saving DataFrame for method 'RF' with exclusion type 'more_restricted':
- Number of features: 57
- Shape of DataFrame: (7890, 58)
- First few columns: ['23', '37', '2_11', '41', '11']...
Success: Saved output/feather_files/predict_finalgrade_RF_more_restricted.feather with FinalGrade as the last column.
Saving DataFrame for method 'Lasso' with exclusion type 'more_restricted':
- Number of features: 61
- Shape of DataFrame: (7890, 62)
- First few columns: ['23', '37', '133', '41', '11']...
Success: Saved output/feather_files/predict_finalgrade_Lasso_more_restricted.feather with FinalGrade as the last column.
Saving DataFrame for method 'FR' with exclusion type 'restricted':
- Number of fe

In [ ]:
doc.add_heading('Selected Features', 3)
doc.add_heading('More Restricted Results', 4)
print("For predicting 'Passed'")
print(extract_common_features(selected_features_predict_passed_more_restricted))
print(extract_common_features(combined_features_passed_more_restricted))
print("For predicting 'FinalGrade' from students that passed")
print(extract_common_features(selected_features_finalgrade_more_restricted))
print(extract_common_features(combined_features_finalgrade_more_restricted))
print("For predicting 'Total_AP'")
print(extract_common_features(selected_features_total_ap_more_restricted))
print(extract_common_features(combined_features_total_ap_regression_more_restricted))
print("For predicting 'Total_PP'")
print(extract_common_features(selected_features_total_pp_more_restricted))
print(extract_common_features(combined_features_total_pp_regression_more_restricted))
print("For predicting 'Total_SPA'")
print(extract_common_features(selected_features_total_spa_more_restricted))
print(extract_common_features(combined_features_total_spa_regression_more_restricted))
print("")
doc.add_heading('Restricted Results', 4)
print("For predicting 'Passed'")
print(extract_common_features(selected_features_predict_passed_restricted))
print(extract_common_features(combined_features_passed_restricted))
print("For predicting 'FinalGrade' from students that passed")
print(extract_common_features(selected_features_finalgrade_restricted))
print(extract_common_features(combined_features_finalgrade_restricted))
print("For predicting 'Total_AP'")
print(extract_common_features(selected_features_total_ap_restricted))
print(extract_common_features(combined_features_total_ap_restricted))
print("For predicting 'Total_PP'")
print(extract_common_features(selected_features_total_pp_restricted))
print(extract_common_features(combined_features_total_pp_restricted))
print("For predicting 'Total_SPA'")
print(extract_common_features(selected_features_total_spa_restricted))
print(extract_common_features(combined_features_total_spa_restricted))
print("")
doc.add_heading('Less Restricted Results', 4)
print("For predicting 'Passed'")
print(extract_common_features(selected_features_predict_passed_less_restricted))
print(extract_common_features(combined_features_passed_less_restricted))
print("For predicting 'FinalGrade' from students that passed")
print(extract_common_features(selected_features_finalgrade_less_restricted))
print(extract_common_features(combined_features_finalgrade_less_restricted))
print("For predicting 'Total_AP'")
print(extract_common_features(selected_features_total_ap_less_restricted))
print(extract_common_features(combined_features_total_ap_less_restricted))
print("For predicting 'Total_PP'")
print(extract_common_features(selected_features_total_pp_less_restricted))
print(extract_common_features(combined_features_total_pp_less_restricted))
print("For predicting 'Total_SPA'")
print(extract_common_features(selected_features_total_spa_less_restricted))
print(extract_common_features(combined_features_total_spa_less_restricted))

For predicting 'Passed'
{'MI': ['38', '31', '34', '42', '10', '35', '30', '50', '20', '12', '32'], 'RF': ['31', '10', '35', '12', '30'], 'DT': []}
{'MI': [], 'RF': [], 'DT': []}
For predicting 'FinalGrade' from students that passed
{'MI': [], 'RF': ['38', '34', '31', '11', '35', '10', '30', '50', '32'], 'DT': []}
{'MI': [], 'RF': [], 'DT': []}
For predicting 'Total_AP'
{'MI': ['38', '31', '34', '42', '10', '35', '30', '50', '20', '12', '32', '1', '36'], 'RF': ['38', '31', '34', '32', '11', '35', '10', '12', '30'], 'DT': ['38', '32', '34', '30']}
{'MI': [], 'RF': [], 'DT': []}
For predicting 'Total_PP'
{'MI': ['38', '31', '34', '42', '32', '10', '35', '50', '12', '30', '1', '36'], 'RF': ['38', '34', '31', '11', '35', '10', '30', '32'], 'DT': ['38', '34', '31', '32', '11', '35', '10', '30']}
{'MI': [], 'RF': [], 'DT': []}
For predicting 'Total_SPA'
{'MI': ['34', '31', '42', '10', '35', '30', '12', '32', '1', '36'], 'RF': ['38', '31', '34', '32', '11', '35', '10', '30'], 'DT': ['38', '31'

In [ ]:
print(get_activity_name_by_id(38, dataset_params, 2016))
print(get_activity_name_by_id(31, dataset_params, 2016))
print(get_activity_name_by_id(34, dataset_params, 2016))
print(get_activity_name_by_id(42, dataset_params, 2016))
print(get_activity_name_by_id(10, dataset_params, 2016))
print(get_activity_name_by_id(35, dataset_params, 2016))
print(get_activity_name_by_id(30, dataset_params, 2016))
print(get_activity_name_by_id(50, dataset_params, 2016))
print(get_activity_name_by_id(20, dataset_params, 2016))
print(get_activity_name_by_id(12, dataset_params, 2016))
print(get_activity_name_by_id(32, dataset_params, 2016))

Parking
Maria's dashboard
Wolis
Leading is ...
Analysis of 3 CVs of candidates
4x100m freestyle
SMART goals
Assessment of the week
Assessment of the week
Psychotechnical test (Moodle)
Submission of CV on Moodle


In [ ]:
# Save the document at the end
save_document('feature_selection_output.docx')

Document saved as feature_selection_output.docx


# 6. Clustering Analysis

In [ ]:
doc = Document()
doc.add_heading('Output of Clustering Report', 0)

## 6.1 Methods

In [ ]:
# Elbow Method for determining the optimal number of clusters
def plot_elbow_method(X, year, max_clusters=10, save_path='Charts/Elbow'):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    distortions = []
    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, random_state=0, n_init=10)
        kmeans.fit(X)
        distortions.append(kmeans.inertia_)

    plt.figure()
    plt.plot(range(1, max_clusters + 1), distortions, marker='o')
    plt.title(f'Elbow Method for Optimal Number of Clusters ({year})')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Distortion')
    plt.savefig(f'{save_path}/Elbow_Method_{year}.png')
    plt.close()

# Display PCA components and their contribution to the Principal Components
def display_pca_components(pca, feature_names):
    components_df = pd.DataFrame(pca.components_, columns=feature_names, index=[f'PC{i+1}' for i in range(pca.n_components_)])
    explained_variance = pca.explained_variance_ratio_
    print("PCA Components and their contribution to the Principal Components:")
    print(components_df)
    print("\nExplained variance ratio per Principal Component:")
    for i, variance in enumerate(explained_variance):
        print(f"PC{i+1}: {variance:.4f}")

# Perform K-Means clustering with optional PCA
def perform_kmeans_clustering(df, features, year, num_clusters, use_pca=True, save_path='Charts/Clustering'):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Data preparation
    X = df[features].values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    if use_pca:
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_scaled)
        display_pca_components(pca, features)

    # K-Means Clustering
    kmeans = KMeans(n_clusters=num_clusters, init='k-means++', random_state=0, n_init=10)
    y_kmeans = kmeans.fit_predict(X_scaled)

    # Evaluate clustering
    silhouette_avg = silhouette_score(X_scaled, y_kmeans)
    davies_bouldin = davies_bouldin_score(X_scaled, y_kmeans)
    calinski_harabasz = calinski_harabasz_score(X_scaled, y_kmeans)

    print(f"Silhouette Score for {num_clusters} clusters: {silhouette_avg:.3f}")
    print(f"Davies-Bouldin Index for {num_clusters} clusters: {davies_bouldin:.3f}")
    print(f"Calinski-Harabasz Index for {num_clusters} clusters: {calinski_harabasz:.3f}")

    # Plotting the clusters with PCA-reduced data if PCA is used
    if use_pca:
        centroids = kmeans.cluster_centers_
        centroids_pca = pca.transform(centroids)
        plt.figure()
        colors = ['red', 'green', 'blue', 'purple', 'orange', 'brown']  # Extend this list if needed
        for i in range(num_clusters):
            plt.scatter(X_pca[y_kmeans == i, 0], X_pca[y_kmeans == i, 1], color=colors[i % len(colors)], label=f'Cluster {i+1}')
        plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], s=100, c='yellow', label='Centroids', marker='x')
        plt.title(f'PCA-Reduced K-Means Clustering of {num_clusters} Clusters ({year})')
        plt.xlabel('Principal Component 1')
        plt.ylabel('Principal Component 2')
        plt.legend()
        plt.savefig(f'{save_path}/PCA_KMeans_{num_clusters}_Clusters_{year}.png')
        plt.close()
    elif X_scaled.shape[1] == 2:  # If no PCA and only 2 features, plot directly
        plt.figure()
        colors = ['red', 'green', 'blue', 'purple', 'orange', 'brown']
        for i in range(num_clusters):
            plt.scatter(X_scaled[y_kmeans == i, 0], X_scaled[y_kmeans == i, 1], color=colors[i % len(colors)], label=f'Cluster {i+1}')
        plt.title(f'K-Means Clustering of {num_clusters} Clusters ({year})')
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.legend()
        plt.savefig(f'{save_path}/KMeans_{num_clusters}_Clusters_{year}.png')
        plt.close()

# Perform Hierarchical Clustering with optional PCA
def perform_hierarchical_clustering(df, features, year, num_clusters, use_pca=True, save_path='output/Clustering'):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    X = df[features].values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    if use_pca:
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_scaled)
        display_pca_components(pca, features)

    # Hierarchical Clustering
    hc = AgglomerativeClustering(n_clusters=num_clusters, linkage='ward')
    y_hc = hc.fit_predict(X_scaled)

    # Evaluate clustering
    silhouette_avg = silhouette_score(X_scaled, y_hc)
    davies_bouldin = davies_bouldin_score(X_scaled, y_hc)
    calinski_harabasz = calinski_harabasz_score(X_scaled, y_hc)

    print(f"Silhouette Score for {num_clusters} clusters: {silhouette_avg:.3f}")
    print(f"Davies-Bouldin Index for {num_clusters} clusters: {davies_bouldin:.3f}")
    print(f"Calinski-Harabasz Index for {num_clusters} clusters: {calinski_harabasz:.3f}")

    # Plotting the clusters with PCA-reduced data if PCA is used
    if use_pca:
        plt.figure()
        colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown']  # Extend this list if needed
        for i in range(num_clusters):
            plt.scatter(X_pca[y_hc == i, 0], X_pca[y_hc == i, 1], color=colors[i % len(colors)], label=f'Cluster {i+1}')
        plt.title(f'PCA-Reduced Hierarchical Clustering of Students ({year})')
        plt.xlabel('Principal Component 1')
        plt.ylabel('Principal Component 2')
        plt.legend()
        plt.savefig(f'{save_path}/PCA_Hierarchical_Clusters_{year}.png')
        plt.close()

        # Plotting the Dendrogram without labels on the x-axis
        plt.figure()
        dendrogram = sch.dendrogram(sch.linkage(X_scaled, method='ward'), no_labels=True)
        plt.title(f'Dendrogram for Hierarchical Clustering ({year})')
        plt.xlabel('Students')
        plt.ylabel('Euclidean distances')
        plt.savefig(f'{save_path}/Dendrogram_{year}.png')
        plt.close()

        # Plotting centroids for Hierarchical Clustering
        cluster_centers = np.array([X_scaled[y_hc == i].mean(axis=0) for i in range(num_clusters)])
        centroids_pca = pca.transform(cluster_centers)
        plt.figure()
        for i in range(num_clusters):
            plt.scatter(X_pca[y_hc == i, 0], X_pca[y_hc == i, 1], color=colors[i % len(colors)], label=f'Cluster {i+1}')
        plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], s=100, c='yellow', label='Cluster Centers', marker='x')
        plt.title(f'PCA-Reduced Hierarchical Clustering with Cluster Centers ({year})')
        plt.xlabel('Principal Component 1')
        plt.ylabel('Principal Component 2')
        plt.legend()
        plt.savefig(f'{save_path}/PCA_Hierarchical_Centers_{year}.png')
        plt.close()
    elif X_scaled.shape[1] == 2:  # If no PCA and only 2 features, plot directly
        plt.figure()
        colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown']
        for i in range(num_clusters):
            plt.scatter(X_scaled[y_hc == i, 0], X_scaled[y_hc == i, 1], color=colors[i % len(colors)], label=f'Cluster {i+1}')
        plt.title(f'Hierarchical Clustering of Students ({year})')
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.legend()
        plt.savefig(f'{save_path}/Hierarchical_Clusters_{year}.png')
        plt.close()

## 6.2. Methods Application

### 6.3.1. Clustering Features: Totals

In [ ]:
doc.add_heading('Clustering Features: Totals', 1)
doc.add_heading('For each year', 2)

In [ ]:
for year in dataset_years:
    clustering_features_totals = [
        col for col in all_lists[str(year)]['created_columns']['TotalColumns']
    ]

    # Prepare the data for clustering
    if all(feature in datasets[year].columns for feature in clustering_features_totals):
        available_features = [feature for feature in clustering_features_totals if feature in datasets[year].columns]
        df = datasets[year][available_features]
        X = StandardScaler().fit_transform(df)

        # Apply Elbow Method
        print(f"Applying Elbow Method for year {year}")
        plot_elbow_method(X, year)

        # Perform clustering and PCA
        for n_clusters in [2, 3, 4]:
            print(f"Running KMeans for {n_clusters} clusters in {year}")
            perform_kmeans_clustering(datasets[year], available_features, year, n_clusters, use_pca=False)

            print(f"Running Hierarchical Clustering for {n_clusters} clusters in {year}")
            perform_hierarchical_clustering(datasets[year], available_features, year, n_clusters, use_pca=False)
    else:
        print(f"Skipping year {year}: Required columns not found in dataset.")


Applying Elbow Method for year 2016
Running KMeans for 2 clusters in 2016
Silhouette Score for 2 clusters: 0.809
Davies-Bouldin Index for 2 clusters: 0.322
Calinski-Harabasz Index for 2 clusters: 1542.388
Running Hierarchical Clustering for 2 clusters in 2016
Silhouette Score for 2 clusters: 0.819
Davies-Bouldin Index for 2 clusters: 0.141
Calinski-Harabasz Index for 2 clusters: 1512.526
Running KMeans for 3 clusters in 2016
Silhouette Score for 3 clusters: 0.836
Davies-Bouldin Index for 3 clusters: 0.279
Calinski-Harabasz Index for 3 clusters: 2665.552
Running Hierarchical Clustering for 3 clusters in 2016
Silhouette Score for 3 clusters: 0.836
Davies-Bouldin Index for 3 clusters: 0.213
Calinski-Harabasz Index for 3 clusters: 2578.661
Running KMeans for 4 clusters in 2016
Silhouette Score for 4 clusters: 0.419
Davies-Bouldin Index for 4 clusters: 0.625
Calinski-Harabasz Index for 4 clusters: 2750.951
Running Hierarchical Clustering for 4 clusters in 2016
Silhouette Score for 4 cluster

In [ ]:
dataset_years = datasets.keys()
for year in dataset_years:
    clustering_features_totals = [
        col for col in all_lists[str(year)]['created_columns']['TotalColumns']
    ]

    # Prepare the data for clustering
    if all(feature in datasets[year].columns for feature in clustering_features_totals):
        available_features = [feature for feature in clustering_features_totals if feature in datasets[year].columns]
        df = datasets[year][available_features]
        X = StandardScaler().fit_transform(df)

        # Apply Elbow Method
        print(f"Applying Elbow Method for year {year}")
        plot_elbow_method(X, year)

        # Perform clustering and PCA
        for n_clusters in [2, 3, 4]:
            print(f"Running KMeans for {n_clusters} clusters in {year}")
            perform_kmeans_clustering(datasets[year], available_features, year, n_clusters, use_pca=True)

            print(f"Running Hierarchical Clustering for {n_clusters} clusters in {year}")
            perform_hierarchical_clustering(datasets[year], available_features, year, n_clusters, use_pca=True)
    else:
        print(f"Skipping year {year}: Required columns not found in dataset.")


Applying Elbow Method for year 2016
Running KMeans for 2 clusters in 2016
PCA Components and their contribution to the Principal Components:
     Total_AP  Total_PP  Total_SPA  Total_Bon
PC1 -0.582978 -0.580650  -0.568298   0.004496
PC2 -0.030514  0.015002   0.023879   0.999136

Explained variance ratio per Principal Component:
PC1: 0.6786
PC2: 0.2504
Silhouette Score for 2 clusters: 0.809
Davies-Bouldin Index for 2 clusters: 0.322
Calinski-Harabasz Index for 2 clusters: 1542.388
Running Hierarchical Clustering for 2 clusters in 2016
PCA Components and their contribution to the Principal Components:
     Total_AP  Total_PP  Total_SPA  Total_Bon
PC1 -0.582978 -0.580650  -0.568298   0.004496
PC2 -0.030514  0.015002   0.023879   0.999136

Explained variance ratio per Principal Component:
PC1: 0.6786
PC2: 0.2504
Silhouette Score for 2 clusters: 0.819
Davies-Bouldin Index for 2 clusters: 0.141
Calinski-Harabasz Index for 2 clusters: 1512.526
Running KMeans for 3 clusters in 2016
PCA Compone

In [ ]:
doc.add_heading('For dataset_complete with no missing data', 2)

In [ ]:
# Prepare the data for clustering
if all(feature in dataset_complete.columns for feature in clustering_features_totals):
    available_features = [feature for feature in clustering_features_totals if feature in dataset_complete.columns]
    df = dataset_complete[available_features]
    X = StandardScaler().fit_transform(df)

    # Apply Elbow Method
    print(f"Applying Elbow Method for dataset_complete with no Missing Data")
    plot_elbow_method(X, year)

    # Perform clustering and PCA
    for n_clusters in [2, 3, 4]:
        print(f"Running KMeans for {n_clusters} clusters for dataset_complete")
        perform_kmeans_clustering(dataset_complete, available_features, year, n_clusters, use_pca=True)

        print(f"Running Hierarchical Clustering for {n_clusters} clusters for dataset_complete")
        perform_hierarchical_clustering(dataset_complete, available_features, year, n_clusters, use_pca=True)
else:
    print(f"Skipping year {year}: Required columns not found in dataset.")

Applying Elbow Method for dataset_complete with no Missing Data
Running KMeans for 2 clusters for dataset_complete
PCA Components and their contribution to the Principal Components:
     Total_AP  Total_PP  Total_SPA  Total_Bon
PC1 -0.618094 -0.692477  -0.366236  -0.065634
PC2 -0.065837  0.003859  -0.074505   0.995037

Explained variance ratio per Principal Component:
PC1: 0.4729
PC2: 0.2500
Silhouette Score for 2 clusters: 0.662
Davies-Bouldin Index for 2 clusters: 0.492
Calinski-Harabasz Index for 2 clusters: 4546.664
Running Hierarchical Clustering for 2 clusters for dataset_complete
PCA Components and their contribution to the Principal Components:
     Total_AP  Total_PP  Total_SPA  Total_Bon
PC1 -0.618094 -0.692477  -0.366236  -0.065634
PC2 -0.065837  0.003859  -0.074505   0.995037

Explained variance ratio per Principal Component:
PC1: 0.4729
PC2: 0.2500
Silhouette Score for 2 clusters: 0.660
Davies-Bouldin Index for 2 clusters: 0.469
Calinski-Harabasz Index for 2 clusters: 4474

In [ ]:
doc.add_heading('All Columns of dataset_complete', 2)

In [ ]:
dataset_complete.columns

Index(['Course', 'Class', 'Sex', '1', '12', '20', '10', '11', '32', '38', '42',
       '35', '50', '30', '31', '36', '34', '72', 'SPA1', 'SPA2', 'SPA3', 'Bon',
       'PP1_1', 'PP1_2', 'PP1_3', 'PP2_1', 'PP2_2', 'PP2_3', 'PP3_1', 'PP3_2',
       'PP3_3', 'PTP1', 'PTP2', 'PTP3', 'FinalGradeInteger', 'Total_AP',
       'Total_PP', 'Total_SPA', 'Total_Bon', 'Year'],
      dtype='object')

In [ ]:
get_activity_name_by_id(1, dataset_params, 2024)

'Submission of CV on Moodle'

In [ ]:
get_activity_name_by_id(12, dataset_params, 2024)

'Psychotechnical test (Moodle)'

In [ ]:
get_activity_name_by_id(20, dataset_params, 2024)

'Assessment of the week'

In [ ]:
get_activity_name_by_id(10, dataset_params, 2024)

'Analysis of 3 CVs of candidates'

In [ ]:
get_activity_name_by_id(11, dataset_params, 2024)

'Presentation on the analyzed CVs'

In [ ]:
get_activity_name_by_id(32, dataset_params, 2024)

'Project execution'

In [ ]:
get_activity_name_by_id(38, dataset_params, 2024)

'Parking'

In [ ]:
get_activity_name_by_id(42, dataset_params, 2024)

'Leading is ...'

In [ ]:
get_activity_name_by_id(35, dataset_params, 2024)

'4x100m freestyle'

In [ ]:
get_activity_name_by_id(50, dataset_params, 2024)

'Assessment of the week'

In [ ]:
get_activity_name_by_id(30, dataset_params, 2024)

'SMART goals'

In [ ]:
get_activity_name_by_id(31, dataset_params, 2024)

"Maria's dashboard"

In [ ]:
get_activity_name_by_id(36, dataset_params, 2024)

'Forced landing - individual'

In [ ]:
get_activity_name_by_id(34, dataset_params, 2024)

'Wolis'

In [ ]:
get_activity_name_by_id(72, dataset_params, 2024)

'Ethical dilemmas'

In [ ]:
doc.add_heading('Results', 3)

clustering_features_complete = [
    col for col in dataset_complete.columns
    if col not in all_lists[str(2024)]['basic_lists']['listCategorical']
]

# Prepare the data for clustering
if all(feature in dataset_complete.columns for feature in clustering_features_complete):
    available_features = [feature for feature in clustering_features_complete if feature in dataset_complete.columns]
    df = dataset_complete[available_features]
    X = StandardScaler().fit_transform(df)

    # Apply Elbow Method
    print(f"Applying Elbow Method for dataset_complete with no Missing Data")
    plot_elbow_method(X, year)

    # Perform clustering and PCA
    for n_clusters in [2, 3, 4]:
        print(f"Running KMeans for {n_clusters} clusters for dataset_complete")
        perform_kmeans_clustering(dataset_complete, available_features, year, n_clusters, use_pca=True)

        print(f"Running Hierarchical Clustering for {n_clusters} clusters for dataset_complete")
        perform_hierarchical_clustering(dataset_complete, available_features, year, n_clusters, use_pca=True)
else:
    print(f"Skipping year {year}: Required columns not found in dataset.")

Applying Elbow Method for dataset_complete with no Missing Data
Running KMeans for 2 clusters for dataset_complete
PCA Components and their contribution to the Principal Components:
            1        12        20        10        11        32        38  \
PC1 -0.179745 -0.202526 -0.179495 -0.178856 -0.030966 -0.172219 -0.163184   
PC2 -0.023103 -0.083435 -0.080291 -0.050100 -0.157664 -0.070887 -0.093581   

           42        35        50  ...     PP3_3      PTP1      PTP2  \
PC1 -0.195360 -0.190516 -0.153512  ... -0.156435 -0.150852 -0.142019   
PC2 -0.091478 -0.005609 -0.118530  ... -0.055284 -0.026712 -0.057104   

         PTP3  FinalGradeInteger  Total_AP  Total_PP  Total_SPA  Total_Bon  \
PC1 -0.127102          -0.260779 -0.208512 -0.235982  -0.159710  -0.005578   
PC2 -0.047740          -0.054497 -0.226406 -0.074916   0.348729  -0.013166   

         Year  
PC1  0.057130  
PC2 -0.417744  

[2 rows x 37 columns]

Explained variance ratio per Principal Component:
PC1: 0.3427


In [ ]:


clustering_features_complete = [
    col for col in dataset_complete.columns
    if col not in all_lists[str(2024)]['basic_lists']['listCategorical']
]

# Prepare the data for clustering
if all(feature in dataset_complete.columns for feature in clustering_features_complete):
    available_features = [feature for feature in clustering_features_complete if feature in dataset_complete.columns]
    df = dataset_complete[available_features]
    X = StandardScaler().fit_transform(df)

    # Apply Elbow Method
    print(f"Applying Elbow Method for dataset_complete with no Missing Data")
    plot_elbow_method(X, year)

    # Perform clustering and PCA
    for n_clusters in [2, 3, 4]:
        print(f"Running KMeans for {n_clusters} clusters for dataset_complete")
        perform_kmeans_clustering(dataset_complete, available_features, year, n_clusters, use_pca=False)

        print(f"Running Hierarchical Clustering for {n_clusters} clusters for dataset_complete")
        perform_hierarchical_clustering(dataset_complete, available_features, year, n_clusters, use_pca=False)
else:
    print(f"Skipping year {year}: Required columns not found in dataset.")

Applying Elbow Method for dataset_complete with no Missing Data
Running KMeans for 2 clusters for dataset_complete
Silhouette Score for 2 clusters: 0.552
Davies-Bouldin Index for 2 clusters: 0.567
Calinski-Harabasz Index for 2 clusters: 2828.071
Running Hierarchical Clustering for 2 clusters for dataset_complete
Silhouette Score for 2 clusters: 0.562
Davies-Bouldin Index for 2 clusters: 0.430
Calinski-Harabasz Index for 2 clusters: 2670.986
Running KMeans for 3 clusters for dataset_complete
Silhouette Score for 3 clusters: 0.237
Davies-Bouldin Index for 3 clusters: 1.439
Calinski-Harabasz Index for 3 clusters: 2601.654
Running Hierarchical Clustering for 3 clusters for dataset_complete
Silhouette Score for 3 clusters: 0.209
Davies-Bouldin Index for 3 clusters: 1.598
Calinski-Harabasz Index for 3 clusters: 2250.285
Running KMeans for 4 clusters for dataset_complete
Silhouette Score for 4 clusters: 0.248
Davies-Bouldin Index for 4 clusters: 1.310
Calinski-Harabasz Index for 4 clusters: 2

### 6.3.2. Clustering Features: Activities Results (AP per activity)

In [ ]:
doc.add_heading('Clustering Features: Week1 and Week2 Activities', 1)

In [ ]:
dataset_years = datasets.keys()
clustering_features = {}
for year in dataset_years:
    clustering_features[year] = [
        col for col in all_lists[str(year)]['basic_lists']['listSyncNumericColumns']
        if col not in all_lists[str(year)]['created_columns']['TotalColumns']
        and col not in ['Week3_AP', 'Week4_AP', 'Week5_AP']
        and col not in all_lists[str(year)]['created_columns']['WeeklyPP']
        and col not in all_lists[str(year)]['created_columns']['OtherColumns']
        and col not in all_lists[str(year)]['basic_lists']['listPPointsColumns']
        and col not in all_lists[str(year)]['basic_lists']['listBonColumns']
        and col not in all_lists[str(year)]['basic_lists']['listSPAColumns']
        and col not in all_lists[str(year)]['basic_lists']['listGrades']
        and col not in all_lists[str(year)]['weekly_and_participation_lists']['listWeek3']
        and col not in all_lists[str(year)]['weekly_and_participation_lists']['listWeek4']
        and col not in all_lists[str(year)]['weekly_and_participation_lists']['listWeek5']
    ]

#### Hierarchichal Clustering Analysis with PCA Reduction

In [ ]:
doc.add_heading('Hierarchichal Clustering Analysis with PCA Reduction', 2)

In [ ]:
for year in dataset_years:
    print(f"Hierarchical Clustering Analysis for {year}")

    missing_features = [feature for feature in clustering_features[year] if feature not in datasets[year].columns]

    if not missing_features:
        available_features = [feature for feature in clustering_features[year] if feature in datasets[year].columns]
        for n_clusters in [2, 3, 4]:
            print(f"Running Hierarchical Clustering for {n_clusters} clusters in {year}")
            perform_hierarchical_clustering(datasets[year], available_features, year, n_clusters)
    else:
        print(f"Skipping year {year}: Required columns not found in dataset.")
        print(f"Missing features for year {year}: {missing_features}")
        #print(f"Available columns {year}: {datasets[year].columns}")


Hierarchical Clustering Analysis for 2016
Running Hierarchical Clustering for 2 clusters in 2016
PCA Components and their contribution to the Principal Components:
            1         3        24        12        20        22         4  \
PC1 -0.179904 -0.012209 -0.183065 -0.200765 -0.190110 -0.037618 -0.009857   
PC2 -0.056332 -0.038523 -0.137883 -0.144589 -0.315897 -0.109929 -0.038128   

           25        10        11  ...       31        52        28        53  \
PC1 -0.202083 -0.167513 -0.043020  ... -0.17933 -0.209058 -0.180563 -0.158269   
PC2 -0.114724 -0.048462 -0.024662  ...  0.17076  0.182594  0.007046 -0.021394   

           36        55        56        34  Week1_AP  Week2_AP  
PC1 -0.221647 -0.220566 -0.220441 -0.195696 -0.216606 -0.251935  
PC2  0.206367  0.206129  0.206020  0.199742 -0.204952  0.195830  

[2 rows x 34 columns]

Explained variance ratio per Principal Component:
PC1: 0.3869
PC2: 0.0672
Silhouette Score for 2 clusters: 0.600
Davies-Bouldin Index for 

#### K-Means Clustering Analysis with PCA Reduction

In [ ]:
doc.add_heading('K-Means Clustering Analysis with PCA Reduction', 2)

In [ ]:
for year in dataset_years:

    # Prepare the data for clustering
    if all(feature in datasets[year].columns for feature in clustering_features[year]):
        available_features = [feature for feature in clustering_features[year] if feature in datasets[year].columns]
        df = datasets[year][available_features]
        X = StandardScaler().fit_transform(df)

        # Apply Elbow Method
        print(f"Applying Elbow Method for year {year}")
        plot_elbow_method(X, year)

        # Perform clustering and PCA
        for n_clusters in [2, 3, 4]:
            print(f"Running KMeans for {n_clusters} clusters in {year}")
            perform_kmeans_clustering(datasets[year], available_features, year, n_clusters)
    else:
        print(f"Skipping year {year}: Required columns not found in dataset.")

Applying Elbow Method for year 2016
Running KMeans for 2 clusters in 2016
PCA Components and their contribution to the Principal Components:
            1         3        24        12        20        22         4  \
PC1 -0.179904 -0.012209 -0.183065 -0.200765 -0.190110 -0.037618 -0.009857   
PC2 -0.056304 -0.038610 -0.137708 -0.144458 -0.315856 -0.110085 -0.038334   

           25        10       11  ...        31        52        28        53  \
PC1 -0.202083 -0.167513 -0.04302  ... -0.179330 -0.209058 -0.180563 -0.158269   
PC2 -0.114773 -0.048688 -0.02461  ...  0.170776  0.182590  0.006954 -0.021501   

           36        55        56        34  Week1_AP  Week2_AP  
PC1 -0.221647 -0.220566 -0.220441 -0.195696 -0.216606 -0.251935  
PC2  0.206376  0.206141  0.206029  0.199738 -0.205030  0.195816  

[2 rows x 34 columns]

Explained variance ratio per Principal Component:
PC1: 0.3869
PC2: 0.0672
Silhouette Score for 2 clusters: 0.584
Davies-Bouldin Index for 2 clusters: 0.576
Calin

#### Clustering analysis of dataset_complete with no missing data

In [ ]:
doc.add_heading('Clustering analysis of dataset_complete with no missing data', 3)

clustering_features_complete = [
    col for col in dataset_complete.columns
    if col not in all_lists[str(2024)]['created_columns']['TotalColumns']
    and col not in ['Week3_AP', 'Week4_AP', 'Week5_AP']
    and col not in all_lists[str(2024)]['created_columns']['WeeklyPP']
    and col not in all_lists[str(2024)]['created_columns']['OtherColumns']
    and col not in all_lists[str(2024)]['basic_lists']['listPPointsColumns']
    and col not in all_lists[str(2024)]['basic_lists']['listBonColumns']
    and col not in all_lists[str(2024)]['basic_lists']['listSPAColumns']
    and col not in all_lists[str(2024)]['basic_lists']['listGrades']
    and col not in all_lists[str(2024)]['weekly_and_participation_lists']['listWeek3']
    and col not in all_lists[str(2024)]['weekly_and_participation_lists']['listWeek4']
    and dataset_complete[col].dtype in [np.float64, np.int64]  # Ensure the column is numeric
]

# Prepare the data for clustering
if all(feature in dataset_complete.columns for feature in clustering_features_complete):
    available_features = [feature for feature in clustering_features_complete if feature in dataset_complete.columns]
    df = dataset_complete[available_features]
    X = StandardScaler().fit_transform(df)

    # Apply Elbow Method
    print(f"Applying Elbow Method for dataset_complete with no Missing Data")
    plot_elbow_method(X, 'dataset_complete')

    # Perform clustering and PCA
    for n_clusters in [2, 3, 4, 5, 6]:
        print(f"Running KMeans for {n_clusters} clusters for dataset_complete")
        perform_kmeans_clustering(dataset_complete, available_features, 'dataset_complete', n_clusters)

        print(f"Running Hierarchical Clustering for {n_clusters} clusters for dataset_complete")
        perform_hierarchical_clustering(dataset_complete, available_features, 'dataset_complete', n_clusters)
else:
    print(f"Skipping dataset_complete: Required columns not found in dataset.")


Applying Elbow Method for dataset_complete with no Missing Data
Running KMeans for 2 clusters for dataset_complete
PCA Components and their contribution to the Principal Components:
            1        12        20        10        11        32        38  \
PC1 -0.264925 -0.302193 -0.265575 -0.265504 -0.062973 -0.274174 -0.270257   
PC2 -0.056915 -0.050052 -0.033205  0.024873 -0.194736  0.403442 -0.364503   

           42        35        50       30        31        36        34  
PC1 -0.302874 -0.293176 -0.231694 -0.24948 -0.280804 -0.314861 -0.271478  
PC2 -0.327767 -0.267319 -0.200442  0.48831  0.420329 -0.068753  0.141654  

Explained variance ratio per Principal Component:
PC1: 0.4343
PC2: 0.0923
Silhouette Score for 2 clusters: 0.622
Davies-Bouldin Index for 2 clusters: 0.529
Calinski-Harabasz Index for 2 clusters: 3936.276
Running Hierarchical Clustering for 2 clusters for dataset_complete
PCA Components and their contribution to the Principal Components:
            1       

# Save Files

In [ ]:

directory_to_zip = 'Charts'
output_zip_file = 'Charts_clustering.zip'

# Create the zip file
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', directory_to_zip)

# Download the zip file
files.download(output_zip_file)

# Save the document at the end
save_document('clustering_output.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Document saved as clustering_output.docx


In [ ]:
directory_to_zip = 'output'
output_zip_file = 'output_feature_selection_clustering.zip'

# Create the zip file
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', directory_to_zip)

# Download the zip file
files.download(output_zip_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>